# Python Coding Questions for Scientist II - Reservations

This notebook contains Python coding questions relevant for the Uber Reserve Data Scientist role, covering:
- Pandas/NumPy data manipulation
- Statistical analysis
- Machine learning implementation
- Optimization problems
- Time series analysis
- Feature engineering

---


In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries loaded successfully!")


## Question 1: Calculate A/B Test Sample Size

**Difficulty:** Easy  
**Topic:** Statistical Analysis

Write a function to calculate the minimum sample size needed for an A/B test given:
- Baseline conversion rate
- Minimum Detectable Effect (MDE)
- Significance level (alpha)
- Statistical power (1 - beta)

**Formula:**
For two-proportion z-test:
$$n = \frac{(Z_{\alpha/2} + Z_{\beta})^2 \cdot (p_1(1-p_1) + p_2(1-p_2))}{(p_1 - p_2)^2}$$

where $p_2 = p_1 + \text{MDE}$


In [ ]:
def calculate_sample_size(baseline_rate, mde, alpha=0.05, power=0.8):
    """
    Calculate minimum sample size per group for A/B test.
    
    Parameters:
    -----------
    baseline_rate : float
        Baseline conversion rate (p1)
    mde : float
        Minimum Detectable Effect (absolute difference)
    alpha : float
        Significance level (default 0.05)
    power : float
        Statistical power (default 0.8)
    
    Returns:
    --------
    int : Minimum sample size per group
    """
    from scipy.stats import norm
    
    # Calculate probabilities
    p1 = baseline_rate
    p2 = p1 + mde
    
    # Validate inputs
    if p2 > 1 or p2 < 0 or p1 < 0 or p1 > 1:
        raise ValueError("Probabilities must be between 0 and 1")
    
    # Z-scores
    z_alpha = norm.ppf(1 - alpha/2)  # Two-tailed test
    z_beta = norm.ppf(power)
    
    # Pooled variance estimate
    p_pooled = (p1 + p2) / 2
    pooled_variance = p_pooled * (1 - p_pooled)
    
    # Sample size calculation
    numerator = (z_alpha + z_beta)**2 * (p1 * (1 - p1) + p2 * (1 - p2))
    denominator = (p1 - p2)**2
    
    n = numerator / denominator
    
    return int(np.ceil(n))

# Test the function
baseline = 0.10  # 10% baseline conversion
mde = 0.02  # Want to detect 2% increase (to 12%)
sample_size = calculate_sample_size(baseline, mde)
print(f"Sample size per group: {sample_size}")
print(f"Total sample size: {sample_size * 2}")

# Example for different scenarios
scenarios = [
    (0.05, 0.01, "Small baseline, small effect"),
    (0.10, 0.02, "Medium baseline, medium effect"),
    (0.20, 0.05, "Large baseline, large effect")
]

for baseline, mde, desc in scenarios:
    n = calculate_sample_size(baseline, mde)
    print(f"\n{desc}:")
    print(f"  Baseline: {baseline:.0%}, MDE: {mde:.0%} → Sample size per group: {n:,}")


## Question 2: Analyze Funnel Conversion Rates

**Difficulty:** Medium  
**Topic:** Data Analysis with Pandas

Given a dataset of user interactions through a booking funnel, calculate:
1. Conversion rate at each stage
2. Drop-off rate between stages
3. Identify the stage with highest drop-off

**Funnel stages:** Awareness → Interest → Consideration → Booking → Completion


In [ ]:
# Generate sample data
np.random.seed(42)
n_users = 10000

data = {
    'user_id': range(1, n_users + 1),
    'saw_reserve': np.random.choice([0, 1], n_users, p=[0.3, 0.7]),  # 70% awareness
}

df = pd.DataFrame(data)

# Each stage depends on previous stage
df['clicked_reserve'] = df['saw_reserve'] * np.random.choice([0, 1], n_users, p=[0.4, 0.6])
df['started_booking'] = df['clicked_reserve'] * np.random.choice([0, 1], n_users, p=[0.3, 0.7])
df['completed_booking'] = df['started_booking'] * np.random.choice([0, 1], n_users, p=[0.2, 0.8])
df['completed_trip'] = df['completed_booking'] * np.random.choice([0, 1], n_users, p=[0.1, 0.9])

def analyze_funnel(df):
    """
    Analyze conversion funnel and identify drop-off points.
    
    Returns:
    --------
    dict : Dictionary with funnel metrics
    """
    stages = {
        'Awareness': 'saw_reserve',
        'Interest': 'clicked_reserve',
        'Consideration': 'started_booking',
        'Booking': 'completed_booking',
        'Completion': 'completed_trip'
    }
    
    results = {}
    previous_count = len(df)
    
    for stage_name, stage_col in stages.items():
        current_count = df[stage_col].sum()
        conversion_rate = current_count / previous_count if previous_count > 0 else 0
        drop_off_rate = 1 - conversion_rate if previous_count > 0 else 0
        overall_rate = current_count / len(df)
        
        results[stage_name] = {
            'count': current_count,
            'conversion_rate': conversion_rate,
            'drop_off_rate': drop_off_rate,
            'overall_rate': overall_rate
        }
        
        previous_count = current_count
    
    return results

# Analyze funnel
funnel_results = analyze_funnel(df)

# Display results
print("=" * 60)
print("FUNNEL ANALYSIS")
print("=" * 60)
print(f"{'Stage':<15} {'Count':<10} {'Conv Rate':<12} {'Drop-off':<12} {'Overall Rate':<12}")
print("-" * 60)

prev_count = len(df)
max_dropoff = 0
max_dropoff_stage = None

for stage, metrics in funnel_results.items():
    print(f"{stage:<15} {metrics['count']:<10,} {metrics['conversion_rate']:<12.1%} "
          f"{metrics['drop_off_rate']:<12.1%} {metrics['overall_rate']:<12.1%}")
    
    if metrics['drop_off_rate'] > max_dropoff:
        max_dropoff = metrics['drop_off_rate']
        max_dropoff_stage = stage
    
    prev_count = metrics['count']

print("-" * 60)
print(f"\n⚠️  Highest drop-off stage: {max_dropoff_stage} ({max_dropoff:.1%})")
print(f"💡 Opportunity: Improving {max_dropoff_stage} could increase overall conversion")


## Question 3: Calculate Difference-in-Differences (DiD)

**Difficulty:** Medium  
**Topic:** Causal Inference

Implement a function to calculate Difference-in-Differences estimator for measuring treatment effect.

**DiD Formula:**
$$\text{DiD} = (Y_{T,Post} - Y_{T,Pre}) - (Y_{C,Post} - Y_{C,Pre})$$

where:
- $Y_{T,Post}$: Treatment group after treatment
- $Y_{T,Pre}$: Treatment group before treatment
- $Y_{C,Post}$: Control group after treatment
- $Y_{C,Pre}$: Control group before treatment


In [ ]:
def calculate_did(df):
    """
    Calculate Difference-in-Differences estimator.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Must have columns: 'group' (treatment/control), 'period' (pre/post), 'outcome'
    
    Returns:
    --------
    dict : DiD estimate and component means
    """
    # Calculate means for each group-period combination
    treatment_pre = df[(df['group'] == 'treatment') & (df['period'] == 'pre')]['outcome'].mean()
    treatment_post = df[(df['group'] == 'treatment') & (df['period'] == 'post')]['outcome'].mean()
    control_pre = df[(df['group'] == 'control') & (df['period'] == 'pre')]['outcome'].mean()
    control_post = df[(df['group'] == 'control') & (df['period'] == 'post')]['outcome'].mean()
    
    # Calculate DiD
    treatment_diff = treatment_post - treatment_pre
    control_diff = control_post - control_pre
    did_estimate = treatment_diff - control_diff
    
    return {
        'did_estimate': did_estimate,
        'treatment_pre': treatment_pre,
        'treatment_post': treatment_post,
        'control_pre': control_pre,
        'control_post': control_post,
        'treatment_diff': treatment_diff,
        'control_diff': control_diff
    }

# Generate sample data
np.random.seed(42)
n_obs = 1000

# Simulate driver availability rates
# Treatment group: incentive program
# Control group: no change

data = {
    'group': np.random.choice(['treatment', 'control'], n_obs),
    'period': np.random.choice(['pre', 'post'], n_obs),
    'outcome': 0.0  # driver availability rate
}

df_did = pd.DataFrame(data)

# Simulate outcomes with treatment effect
for idx, row in df_did.iterrows():
    base_rate = 0.6  # Base availability rate
    
    # Treatment effect: +0.15 (15 percentage points)
    treatment_effect = 0.15 if row['group'] == 'treatment' and row['period'] == 'post' else 0
    
    # Time trend: +0.05 (5 percentage points over time)
    time_trend = 0.05 if row['period'] == 'post' else 0
    
    df_did.loc[idx, 'outcome'] = base_rate + treatment_effect + time_trend + np.random.normal(0, 0.1)

# Calculate DiD
did_results = calculate_did(df_did)

# Display results
print("=" * 70)
print("DIFFERENCE-IN-DIFFERENCES ANALYSIS")
print("=" * 70)
print(f"\nTreatment Group:")
print(f"  Pre-period:  {did_results['treatment_pre']:.3f}")
print(f"  Post-period: {did_results['treatment_post']:.3f}")
print(f"  Difference:  {did_results['treatment_diff']:.3f} ({did_results['treatment_diff']:.1%})")

print(f"\nControl Group:")
print(f"  Pre-period:  {did_results['control_pre']:.3f}")
print(f"  Post-period: {did_results['control_post']:.3f}")
print(f"  Difference:  {did_results['control_diff']:.3f} ({did_results['control_diff']:.1%})")

print(f"\n{'='*70}")
print(f"DiD Estimate (Treatment Effect): {did_results['did_estimate']:.3f} ({did_results['did_estimate']:.1%})")
print(f"{'='*70}")

# Interpretation
if did_results['did_estimate'] > 0:
    print(f"\n✓ Treatment had a positive effect of {did_results['did_estimate']:.1%}")
else:
    print(f"\n✗ Treatment had a negative effect of {did_results['did_estimate']:.1%}")


## Question 4: Build a Cancellation Prediction Model

**Difficulty:** Medium-Hard  
**Topic:** Machine Learning

Build a binary classification model to predict trip cancellations with proper feature engineering and evaluation.


In [ ]:
# Generate synthetic trip data
np.random.seed(42)
n_trips = 10000

def generate_trip_data(n):
    """Generate synthetic trip data with cancellation patterns."""
    data = {}
    
    # Basic trip features
    data['trip_distance'] = np.random.exponential(5, n)
    data['lead_time_minutes'] = np.random.exponential(30, n)  # Time until pickup
    data['hour_of_day'] = np.random.randint(0, 24, n)
    data['day_of_week'] = np.random.randint(0, 7, n)
    data['is_weekend'] = (data['day_of_week'] >= 5).astype(int)
    
    # Rider features
    data['rider_history_cancel_rate'] = np.random.beta(2, 20, n)  # Most riders have low cancel rate
    data['rider_trip_count'] = np.random.poisson(10, n)
    
    # Market features
    data['driver_supply'] = np.random.uniform(0.5, 1.5, n)  # Supply index
    data['surge_multiplier'] = np.random.lognormal(0, 0.3, n)
    data['surge_multiplier'] = np.clip(data['surge_multiplier'], 1.0, 3.0)
    
    # Generate cancellation based on features (with some randomness)
    # Higher cancellation probability for:
    # - Longer lead times
    # - Higher historical cancel rate
    # - Low driver supply
    # - Very high surge pricing
    
    cancel_prob = (
        0.05 +  # Base rate
        0.0003 * data['lead_time_minutes'] +  # Longer wait = more cancellation
        2.0 * data['rider_history_cancel_rate'] +  # Historical pattern
        -0.2 * data['driver_supply'] +  # Low supply = more cancellation
        0.05 * (data['surge_multiplier'] - 1.0) +  # High surge = more cancellation
        np.random.normal(0, 0.02, n)  # Random noise
    )
    
    cancel_prob = np.clip(cancel_prob, 0, 1)
    data['cancelled'] = (np.random.random(n) < cancel_prob).astype(int)
    
    return pd.DataFrame(data)

df_trips = generate_trip_data(n_trips)

print("Sample trip data:")
print(df_trips.head())
print(f"\nCancellation rate: {df_trips['cancelled'].mean():.2%}")


In [ ]:
def build_cancellation_model(df):
    """
    Build and evaluate a cancellation prediction model.
    
    Returns:
    --------
    dict : Model and evaluation metrics
    """
    # Feature engineering
    df_model = df.copy()
    
    # Cyclical encoding for time features
    df_model['hour_sin'] = np.sin(2 * np.pi * df_model['hour_of_day'] / 24)
    df_model['hour_cos'] = np.cos(2 * np.pi * df_model['hour_of_day'] / 24)
    df_model['day_sin'] = np.sin(2 * np.pi * df_model['day_of_week'] / 7)
    df_model['day_cos'] = np.cos(2 * np.pi * df_model['day_of_week'] / 7)
    
    # Feature interactions
    df_model['lead_time_hours'] = df_model['lead_time_minutes'] / 60
    df_model['distance_per_lead_time'] = df_model['trip_distance'] / (df_model['lead_time_minutes'] + 1)
    
    # Prepare features
    feature_cols = [
        'trip_distance', 'lead_time_minutes', 'lead_time_hours',
        'rider_history_cancel_rate', 'rider_trip_count',
        'driver_supply', 'surge_multiplier',
        'is_weekend', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
        'distance_per_lead_time'
    ]
    
    X = df_model[feature_cols]
    y = df_model['cancelled']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train model (using Gradient Boosting for better performance)
    model = GradientBoostingRegressor(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    )
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_pred_proba = model.predict(X_test_scaled)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Evaluation metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, zero_division=0),
        'recall': recall_score(y_test, y_pred, zero_division=0),
        'f1': f1_score(y_test, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_test, y_pred_proba)
    }
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    return {
        'model': model,
        'scaler': scaler,
        'metrics': metrics,
        'feature_importance': feature_importance,
        'y_test': y_test,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

# Build model
results = build_cancellation_model(df_trips)

# Display results
print("=" * 60)
print("CANCELLATION PREDICTION MODEL RESULTS")
print("=" * 60)
print("\nModel Performance:")
for metric, value in results['metrics'].items():
    print(f"  {metric.upper()}: {value:.3f}")

print("\nTop 10 Most Important Features:")
print(results['feature_importance'].head(10).to_string(index=False))

# Additional analysis
print(f"\nBaseline (predicting no cancellation): {1 - results['y_test'].mean():.3f}")
print(f"Model improvement over baseline: {results['metrics']['accuracy'] - (1 - results['y_test'].mean()):.3f}")


## Question 5: Implement Greedy Matching Algorithm

**Difficulty:** Medium  
**Topic:** Optimization Algorithms

Implement a greedy algorithm to match riders with drivers, optimizing for minimum wait time.


In [ ]:
def greedy_matching(riders, drivers, distance_matrix):
    """
    Greedy algorithm for rider-driver matching.
    
    Parameters:
    -----------
    riders : list
        List of rider IDs
    drivers : list
        List of driver IDs
    distance_matrix : np.array
        Matrix where [i, j] is distance from rider i to driver j
    
    Returns:
    --------
    dict : Matches {rider_id: driver_id}
    """
    matches = {}
    available_drivers = set(drivers)
    
    # Sort riders by some priority (e.g., wait time - here we use index as proxy)
    # In practice, you'd sort by actual wait time or priority score
    sorted_riders = sorted(riders)
    
    for rider_idx in sorted_riders:
        if not available_drivers:
            break
        
        # Find closest available driver
        rider_pos = riders.index(rider_idx)
        best_driver = None
        min_distance = float('inf')
        
        for driver_idx in available_drivers:
            driver_pos = drivers.index(driver_idx)
            distance = distance_matrix[rider_pos, driver_pos]
            
            if distance < min_distance:
                min_distance = distance
                best_driver = driver_idx
        
        # Match rider with best driver
        if best_driver is not None:
            matches[rider_idx] = best_driver
            available_drivers.remove(best_driver)
    
    return matches

# Generate sample data
np.random.seed(42)
n_riders = 10
n_drivers = 12

# Rider and driver locations (in 2D space)
rider_locations = np.random.rand(n_riders, 2) * 10
driver_locations = np.random.rand(n_drivers, 2) * 10

# Calculate distance matrix (Euclidean distance)
distance_matrix = np.zeros((n_riders, n_drivers))
for i in range(n_riders):
    for j in range(n_drivers):
        distance_matrix[i, j] = np.sqrt(
            np.sum((rider_locations[i] - driver_locations[j])**2)
        )

riders = list(range(n_riders))
drivers = list(range(n_drivers))

# Perform matching
matches = greedy_matching(riders, drivers, distance_matrix)

# Display results
print("=" * 60)
print("GREEDY MATCHING RESULTS")
print("=" * 60)
print(f"\nMatched {len(matches)} out of {n_riders} riders")
print(f"Available drivers: {n_drivers - len(matches)}")

print("\nMatches (Rider → Driver):")
total_distance = 0
for rider, driver in matches.items():
    distance = distance_matrix[rider, driver]
    total_distance += distance
    print(f"  Rider {rider} → Driver {driver} (distance: {distance:.2f})")

print(f"\nAverage match distance: {total_distance / len(matches):.2f}")
print(f"Total distance: {total_distance:.2f}")

# Compare with optimal (Hungarian algorithm would give optimal)
print("\nNote: For small instances, Hungarian algorithm would give optimal solution.")
print("Greedy algorithm is faster but may not be optimal.")


## Question 6: Calculate Population Stability Index (PSI)

**Difficulty:** Medium  
**Topic:** Data Drift Detection

Implement PSI calculation to detect data drift between training and production data.

**PSI Formula:**
$$\text{PSI} = \sum_{i=1}^{n} (P_{\text{actual},i} - P_{\text{expected},i}) \times \ln\left(\frac{P_{\text{actual},i}}{P_{\text{expected},i}}\right)$$

where $P_i$ is the proportion of observations in bucket $i$.


In [ ]:
def calculate_psi(expected, actual, buckets=10):
    """
    Calculate Population Stability Index (PSI) to detect data drift.
    
    Parameters:
    -----------
    expected : np.array or pd.Series
        Expected distribution (training data)
    actual : np.array or pd.Series
        Actual distribution (production data)
    buckets : int
        Number of buckets for binning
    
    Returns:
    --------
    float : PSI value
    """
    # Create bins based on expected data
    min_val = min(expected.min(), actual.min())
    max_val = max(expected.max(), actual.max())
    
    # Handle edge case where all values are the same
    if min_val == max_val:
        return 0.0
    
    # Create buckets
    bucket_edges = np.linspace(min_val, max_val, buckets + 1)
    
    # Bin the data
    expected_counts, _ = np.histogram(expected, bins=bucket_edges)
    actual_counts, _ = np.histogram(actual, bins=bucket_edges)
    
    # Normalize to proportions
    expected_pct = expected_counts / len(expected)
    actual_pct = actual_counts / len(actual)
    
    # Avoid division by zero (add small epsilon)
    epsilon = 1e-10
    expected_pct = expected_pct + epsilon
    actual_pct = actual_pct + epsilon
    
    # Normalize again to ensure sum = 1
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()
    
    # Calculate PSI
    psi = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))
    
    return psi

# Generate sample data
np.random.seed(42)

# Training data (expected distribution)
training_data = np.random.normal(50, 10, 10000)

# Production data with different distributions
prod_data_no_drift = np.random.normal(50, 10, 5000)  # Same distribution
prod_data_shifted = np.random.normal(55, 10, 5000)   # Mean shifted
prod_data_variance = np.random.normal(50, 15, 5000)  # Variance changed

# Calculate PSI
psi_no_drift = calculate_psi(training_data, prod_data_no_drift)
psi_shifted = calculate_psi(training_data, prod_data_shifted)
psi_variance = calculate_psi(training_data, prod_data_variance)

# Display results
print("=" * 70)
print("POPULATION STABILITY INDEX (PSI) ANALYSIS")
print("=" * 70)

print(f"\nPSI Interpretation:")
print(f"  PSI < 0.1:    No significant drift")
print(f"  0.1 ≤ PSI < 0.25: Moderate drift")
print(f"  PSI ≥ 0.25:   Significant drift")

print(f"\nResults:")
print(f"  No drift scenario:     PSI = {psi_no_drift:.4f} ✓")
print(f"  Mean shifted scenario: PSI = {psi_shifted:.4f} ⚠️")
print(f"  Variance changed:      PSI = {psi_variance:.4f} ⚠️")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

scenarios = [
    (prod_data_no_drift, psi_no_drift, "No Drift"),
    (prod_data_shifted, psi_shifted, "Mean Shifted"),
    (prod_data_variance, psi_variance, "Variance Changed")
]

for ax, (prod_data, psi, title) in zip(axes, scenarios):
    ax.hist(training_data, bins=30, alpha=0.5, label='Training', density=True)
    ax.hist(prod_data, bins=30, alpha=0.5, label='Production', density=True)
    ax.set_title(f"{title}\nPSI = {psi:.4f}")
    ax.set_xlabel('Value')
    ax.set_ylabel('Density')
    ax.legend()

plt.tight_layout()
plt.show()


## Question 7: Time Series Feature Engineering

**Difficulty:** Medium  
**Topic:** Feature Engineering

Create time-based features from datetime data, including cyclical encoding, lag features, and rolling statistics.


In [ ]:
def create_time_features(df, date_col='timestamp'):
    """
    Create comprehensive time-based features.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with datetime column
    date_col : str
        Name of datetime column
    
    Returns:
    --------
    pd.DataFrame : DataFrame with additional time features
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Basic datetime features
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day
    df['hour'] = df[date_col].dt.hour
    df['day_of_week'] = df[date_col].dt.dayofweek  # Monday=0, Sunday=6
    df['day_of_year'] = df[date_col].dt.dayofyear
    df['week_of_year'] = df[date_col].dt.isocalendar().week
    
    # Cyclical encoding (captures periodic patterns)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    # Binary flags
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['is_morning'] = ((df['hour'] >= 6) & (df['hour'] < 12)).astype(int)
    df['is_afternoon'] = ((df['hour'] >= 12) & (df['hour'] < 18)).astype(int)
    df['is_evening'] = ((df['hour'] >= 18) & (df['hour'] < 24)).astype(int)
    df['is_night'] = ((df['hour'] >= 0) & (df['hour'] < 6)).astype(int)
    
    # Business hour indicators
    df['is_business_hour'] = ((df['day_of_week'] < 5) & 
                               (df['hour'] >= 9) & 
                               (df['hour'] < 17)).astype(int)
    
    return df

# Generate sample time series data
dates = pd.date_range('2024-01-01', periods=1000, freq='H')
df_time = pd.DataFrame({
    'timestamp': dates,
    'trip_count': np.random.poisson(10, 1000)  # Simulated trip counts
})

# Create time features
df_time_features = create_time_features(df_time)

print("Sample time features:")
print(df_time_features[['timestamp', 'hour', 'day_of_week', 'hour_sin', 
                        'hour_cos', 'is_weekend', 'is_business_hour']].head(10))

print("\nFeature summary:")
print(f"  Total features created: {len(df_time_features.columns) - 2}")  # Exclude original cols
print(f"  Cyclical features: 6 (hour, day, month sin/cos)")
print(f"  Binary flags: 6 (weekend, time periods, business hours)")
print(f"  Basic datetime: 7 (year, month, day, etc.)")


## Question 8: Handle Imbalanced Dataset with SMOTE

**Difficulty:** Medium  
**Topic:** Machine Learning - Class Imbalance

Implement SMOTE (Synthetic Minority Oversampling Technique) to handle imbalanced classification datasets.


In [ ]:
def smote_oversample(X, y, k=5, minority_class=1, oversampling_ratio=1.0):
    """
    Simplified SMOTE implementation.
    
    Parameters:
    -----------
    X : np.array
        Feature matrix
    y : np.array
        Target vector
    k : int
        Number of nearest neighbors
    minority_class : int
        Class to oversample
    oversampling_ratio : float
        Ratio of synthetic samples to create (1.0 = equal to majority class)
    
    Returns:
    --------
    X_resampled, y_resampled : Oversampled data
    """
    from sklearn.neighbors import NearestNeighbors
    
    X = np.array(X)
    y = np.array(y)
    
    # Separate minority and majority classes
    minority_indices = np.where(y == minority_class)[0]
    majority_indices = np.where(y != minority_class)[0]
    
    X_minority = X[minority_indices]
    X_majority = X[majority_indices]
    
    # Calculate how many samples to generate
    n_minority = len(X_minority)
    n_majority = len(X_majority)
    n_synthetic = int((n_majority - n_minority) * oversampling_ratio)
    
    # Find k nearest neighbors for each minority sample
    nn = NearestNeighbors(n_neighbors=k+1)  # +1 because sample is its own neighbor
    nn.fit(X_minority)
    neighbors = nn.kneighbors(X_minority, return_distance=False)
    
    # Generate synthetic samples
    synthetic_samples = []
    for i in range(n_synthetic):
        # Randomly choose a minority sample
        sample_idx = np.random.randint(n_minority)
        sample = X_minority[sample_idx]
        
        # Randomly choose one of its k nearest neighbors (excluding itself)
        neighbor_idx = np.random.choice(neighbors[sample_idx][1:])
        neighbor = X_minority[neighbor_idx]
        
        # Generate synthetic sample along the line between sample and neighbor
        alpha = np.random.random()
        synthetic = sample + alpha * (neighbor - sample)
        synthetic_samples.append(synthetic)
    
    # Combine original and synthetic samples
    X_resampled = np.vstack([X, np.array(synthetic_samples)])
    y_resampled = np.hstack([y, np.full(n_synthetic, minority_class)])
    
    return X_resampled, y_resampled

# Generate imbalanced dataset
np.random.seed(42)
n_samples = 1000

# Features
X = np.random.randn(n_samples, 5)

# Imbalanced target (90% class 0, 10% class 1)
y = np.random.choice([0, 1], n_samples, p=[0.9, 0.1])

print("Original dataset:")
print(f"  Class 0: {np.sum(y == 0)} ({np.mean(y == 0):.1%})")
print(f"  Class 1: {np.sum(y == 1)} ({np.mean(y == 1):.1%})")

# Apply SMOTE
X_resampled, y_resampled = smote_oversample(X, y, k=5, oversampling_ratio=1.0)

print("\nAfter SMOTE:")
print(f"  Class 0: {np.sum(y_resampled == 0)} ({np.mean(y_resampled == 0):.1%})")
print(f"  Class 1: {np.sum(y_resampled == 1)} ({np.mean(y_resampled == 1):.1%})")
print(f"  Total samples: {len(y_resampled)}")
print(f"  Synthetic samples created: {len(y_resampled) - len(y)}")

# Note: In production, use imbalanced-learn library
print("\n💡 Note: For production use, consider using imbalanced-learn library:")
print("   from imblearn.over_sampling import SMOTE")
print("   smote = SMOTE(random_state=42)")
print("   X_resampled, y_resampled = smote.fit_resample(X, y)")


## Question 9: Calculate Cohort Retention

**Difficulty:** Medium  
**Topic:** Business Analytics

Calculate cohort-based retention rates to understand user behavior over time.


In [ ]:
def calculate_cohort_retention(df, user_col='user_id', date_col='date', period='M'):
    """
    Calculate cohort retention rates.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Transaction data with user_id and date
    user_col : str
        Column name for user identifier
    date_col : str
        Column name for date
    period : str
        Period for cohort grouping ('M' for monthly, 'W' for weekly)
    
    Returns:
    --------
    pd.DataFrame : Cohort retention matrix
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Assign cohort based on first transaction
    df['cohort'] = df.groupby(user_col)[date_col].transform('min')
    
    # Create period labels
    df['cohort_period'] = df['cohort'].dt.to_period(period)
    df['period'] = df[date_col].dt.to_period(period)
    
    # Calculate period number (0, 1, 2, ...)
    df['period_number'] = (df['period'] - df['cohort_period']).apply(attrgetter('n'))
    
    # Count unique users per cohort-period
    cohort_data = df.groupby(['cohort_period', 'period_number'])[user_col].nunique().reset_index()
    cohort_data.columns = ['cohort', 'period', 'users']
    
    # Pivot to create retention matrix
    retention_matrix = cohort_data.pivot(index='cohort', columns='period', values='users')
    
    # Calculate retention rates (as percentage of cohort size)
    cohort_sizes = retention_matrix.iloc[:, 0]  # First period = cohort size
    retention_pct = retention_matrix.div(cohort_sizes, axis=0) * 100
    
    return retention_pct

# Generate sample user transaction data
np.random.seed(42)
n_users = 1000
start_date = pd.Timestamp('2024-01-01')
end_date = pd.Timestamp('2024-06-30')

transactions = []
for user_id in range(n_users):
    # User's first transaction date (cohort assignment)
    first_date = start_date + pd.Timedelta(days=np.random.randint(0, 90))
    
    # Number of transactions for this user
    n_transactions = np.random.poisson(5)
    
    # Generate transaction dates (with some retention decay)
    for i in range(n_transactions):
        # Later transactions become less likely
        if np.random.random() < 0.3 ** (i / 3):
            transaction_date = first_date + pd.Timedelta(days=np.random.randint(0, 150))
            if transaction_date <= end_date:
                transactions.append({
                    'user_id': user_id,
                    'date': transaction_date
                })

df_cohort = pd.DataFrame(transactions)

# Calculate retention
retention = calculate_cohort_retention(df_cohort)

print("Cohort Retention Matrix (%):")
print("=" * 80)
print(retention.round(1).fillna(''))

print("\nInterpretation:")
print("- Rows represent cohorts (when users first transacted)")
print("- Columns represent periods since cohort start (0 = first period)")
print("- Values show % of cohort users active in that period")
print("- Higher values in later periods indicate better retention")


## Key Concepts for Data Science Interviews

### Data Structures to Know:
1. **Arrays/Lists**: Basic indexing, slicing, array operations
2. **Hash Maps/Dictionaries**: O(1) lookups, counting frequencies
3. **Sets**: Uniqueness, set operations
4. **Heaps**: Priority queues, top-K elements
5. **Stacks/Queues**: LIFO/FIFO operations
6. **Trees**: Binary trees, BST operations
7. **Graphs**: Adjacency lists, DFS/BFS, shortest paths

### Algorithms to Master:
1. **Two Pointers**: Sliding window, fast/slow pointers
2. **Binary Search**: Search in sorted arrays
3. **Greedy Algorithms**: Local optimization
4. **Dynamic Programming**: Overlapping subproblems
5. **Graph Algorithms**: DFS, BFS, Dijkstra's
6. **Sorting**: Merge sort, quick sort, counting sort

### Time Complexity Basics:
- O(1): Constant time
- O(log n): Logarithmic (binary search)
- O(n): Linear (single pass)
- O(n log n): Linearithmic (sorting)
- O(n²): Quadratic (nested loops)
- O(2^n): Exponential (recursion without memoization)

---


## Question 10: Two Sum Problem

**Difficulty:** Easy  
**Topic:** Arrays, Hash Maps  
**Frequency:** Very Common in Interviews

Given an array of integers and a target sum, find two numbers that add up to the target.

**Key Concept:** Use hash map for O(1) lookups instead of nested loops.


In [ ]:
def two_sum(nums, target):
    """
    Find two numbers in array that sum to target.
    
    Time Complexity: O(n)
    Space Complexity: O(n)
    
    Parameters:
    -----------
    nums : List[int]
        List of integers
    target : int
        Target sum
    
    Returns:
    --------
    List[int]: Indices of two numbers that sum to target
    """
    # Hash map to store value -> index
    num_map = {}
    
    for i, num in enumerate(nums):
        complement = target - num
        
        # Check if complement exists in map
        if complement in num_map:
            return [num_map[complement], i]
        
        # Store current number and index
        num_map[num] = i
    
    return []  # No solution found

# Test cases
test_cases = [
    ([2, 7, 11, 15], 9, [0, 1]),
    ([3, 2, 4], 6, [1, 2]),
    ([3, 3], 6, [0, 1]),
    ([1, 5, 3, 7], 10, [1, 3])
]

print("Two Sum Problem:")
print("=" * 60)
for nums, target, expected in test_cases:
    result = two_sum(nums, target)
    print(f"Input: nums={nums}, target={target}")
    print(f"Output: {result}")
    print(f"Expected: {expected}")
    print(f"Match: {result == expected}")
    print("-" * 60)

# Application: Find two trips with combined distance equal to target
trips = [5.2, 3.1, 8.7, 2.5, 6.3, 4.9]
target_distance = 10.0
result_indices = two_sum([int(d*10) for d in trips], int(target_distance*10))
if result_indices:
    print(f"\nTrips with combined distance {target_distance}:")
    print(f"Trip {result_indices[0]}: {trips[result_indices[0]]} km")
    print(f"Trip {result_indices[1]}: {trips[result_indices[1]]} km")


## Question 11: Sliding Window Maximum (Relevant for Time Series)

**Difficulty:** Medium  
**Topic:** Arrays, Sliding Window, Deque  
**Use Case:** Moving averages, time window aggregations

Find the maximum in each sliding window of size k.


In [ ]:
from collections import deque

def sliding_window_maximum(nums, k):
    """
    Find maximum in each sliding window of size k.
    
    Time Complexity: O(n) - each element added/removed once
    Space Complexity: O(k) - deque size
    
    Key Concept: Use deque to maintain potential maximums in window
    """
    if not nums or k == 0:
        return []
    
    # Deque stores indices of elements
    dq = deque()
    result = []
    
    for i in range(len(nums)):
        # Remove indices outside current window
        while dq and dq[0] <= i - k:
            dq.popleft()
        
        # Remove indices whose values are smaller than current
        # (they can never be maximum in this or future windows)
        while dq and nums[dq[-1]] < nums[i]:
            dq.pop()
        
        # Add current index
        dq.append(i)
        
        # Window of size k is complete
        if i >= k - 1:
            result.append(nums[dq[0]])
    
    return result

# Test
print("Sliding Window Maximum:")
print("=" * 60)
nums = [1, 3, -1, -3, 5, 3, 6, 7]
k = 3
result = sliding_window_maximum(nums, k)
print(f"Input: {nums}, k={k}")
print(f"Output: {result}")
print(f"Explanation: Max in [1,3,-1]=3, [3,-1,-3]=3, [-1,-3,5]=5, etc.")

# Application: Find maximum driver availability in each hour window
hourly_availability = [10, 15, 12, 8, 20, 25, 18, 22, 30, 28]
window_size = 3
max_per_window = sliding_window_maximum(hourly_availability, window_size)
print(f"\nHourly driver availability: {hourly_availability}")
print(f"Maximum in each {window_size}-hour window: {max_per_window}")


## Question 12: Top K Frequent Elements (Common in ML/DS)

**Difficulty:** Medium  
**Topic:** Hash Maps, Heaps  
**Use Case:** Feature importance, most common events, top users

Find K most frequent elements in an array.


In [ ]:
import heapq
from collections import Counter

def top_k_frequent(nums, k):
    """
    Find K most frequent elements.
    
    Time Complexity: O(n + k log n) - Counter + heap operations
    Space Complexity: O(n) - for Counter and heap
    
    Approach 1: Using Counter and heapq.nlargest
    """
    # Count frequencies
    count = Counter(nums)
    
    # Get k largest by frequency
    return [item for item, freq in count.most_common(k)]

def top_k_frequent_heap(nums, k):
    """
    Alternative: Using min heap of size k.
    
    More memory efficient for large datasets with k << n
    """
    count = Counter(nums)
    
    # Min heap of size k
    heap = []
    
    for num, freq in count.items():
        if len(heap) < k:
            heapq.heappush(heap, (freq, num))
        elif freq > heap[0][0]:
            heapq.heapreplace(heap, (freq, num))
    
    # Extract results
    return [num for freq, num in heap]

# Test
print("Top K Frequent Elements:")
print("=" * 60)
nums = [1, 1, 1, 2, 2, 3, 4, 4, 4, 4]
k = 2
result1 = top_k_frequent(nums, k)
result2 = top_k_frequent_heap(nums, k)
print(f"Input: {nums}, k={k}")
print(f"Output (Counter): {result1}")
print(f"Output (Heap): {result2}")

# Application: Find top K cities by trip count
city_trips = ['NYC', 'SF', 'NYC', 'LA', 'SF', 'SF', 'NYC', 'CHI', 'NYC', 'LA']
top_cities = top_k_frequent(city_trips, 3)
print(f"\nCity trip counts: {Counter(city_trips)}")
print(f"Top 3 cities: {top_cities}")


## Question 13: Binary Search (Search in Sorted Array)

**Difficulty:** Easy-Medium  
**Topic:** Binary Search  
**Use Case:** Search in sorted data, finding thresholds

Implement binary search to find target in sorted array.


In [ ]:
def binary_search(arr, target):
    """
    Binary search in sorted array.
    
    Time Complexity: O(log n)
    Space Complexity: O(1)
    
    Key Concept: Divide and conquer, eliminate half of search space each time
    """
    left, right = 0, len(arr) - 1
    
    while left <= right:
        mid = left + (right - left) // 2  # Avoid overflow
        
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    
    return -1  # Not found

def binary_search_first_occurrence(arr, target):
    """
    Find first occurrence of target (useful for duplicates).
    """
    left, right = 0, len(arr)
    result = -1
    
    while left < right:
        mid = left + (right - left) // 2
        if arr[mid] == target:
            result = mid
            right = mid  # Continue searching left
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid
    
    return result

# Test
print("Binary Search:")
print("=" * 60)
sorted_array = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
target = 11
result = binary_search(sorted_array, target)
print(f"Array: {sorted_array}")
print(f"Target: {target}")
print(f"Index: {result}")
print(f"Value at index: {sorted_array[result] if result != -1 else 'Not found'}")

# Application: Find driver with rating >= threshold
driver_ratings = [3.2, 3.5, 3.8, 4.1, 4.3, 4.5, 4.7, 4.9, 5.0]
threshold = 4.5
# Find first index where rating >= threshold
left, right = 0, len(driver_ratings)
while left < right:
    mid = left + (right - left) // 2
    if driver_ratings[mid] < threshold:
        left = mid + 1
    else:
        right = mid
print(f"\nDriver ratings: {driver_ratings}")
print(f"First driver with rating >= {threshold}: index {left}, rating {driver_ratings[left]}")


## Question 14: Valid Parentheses (Stack Problem)

**Difficulty:** Easy  
**Topic:** Stacks  
**Use Case:** Expression validation, nested structures

Check if parentheses in a string are valid (properly matched).


In [ ]:
def is_valid_parentheses(s):
    """
    Check if parentheses are valid.
    
    Time Complexity: O(n)
    Space Complexity: O(n) worst case
    
    Key Concept: Use stack to track opening brackets, pop when matching closing bracket
    """
    stack = []
    mapping = {')': '(', '}': '{', ']': '['}
    
    for char in s:
        if char in mapping:  # Closing bracket
            if not stack or stack.pop() != mapping[char]:
                return False
        else:  # Opening bracket
            stack.append(char)
    
    return len(stack) == 0

# Test
print("Valid Parentheses:")
print("=" * 60)
test_cases = [
    ("()", True),
    ("()[]{}", True),
    ("(]", False),
    ("([)]", False),
    ("{[]}", True),
    ("", True)
]

for s, expected in test_cases:
    result = is_valid_parentheses(s)
    print(f"Input: '{s}' -> Output: {result}, Expected: {expected}, Match: {result == expected}")

# Application: Validate nested JSON-like structure for configuration
config_str = "{[{()}]}"
print(f"\nConfig validation '{config_str}': {is_valid_parentheses(config_str)}")


## Question 15: Longest Substring Without Repeating Characters

**Difficulty:** Medium  
**Topic:** Sliding Window, Hash Maps  
**Use Case:** Finding unique sequences, data deduplication

Find length of longest substring without repeating characters.


In [ ]:
def length_of_longest_substring(s):
    """
    Find length of longest substring without repeating characters.
    
    Time Complexity: O(n)
    Space Complexity: O(min(n, m)) where m is charset size
    
    Key Concept: Sliding window with hash map to track character positions
    """
    char_map = {}  # character -> last seen index
    start = 0
    max_length = 0
    
    for end, char in enumerate(s):
        # If character seen before and within current window
        if char in char_map and char_map[char] >= start:
            start = char_map[char] + 1
        
        char_map[char] = end
        max_length = max(max_length, end - start + 1)
    
    return max_length

# Test
print("Longest Substring Without Repeating Characters:")
print("=" * 60)
test_cases = [
    ("abcabcbb", 3),  # "abc"
    ("bbbbb", 1),     # "b"
    ("pwwkew", 3),    # "wke"
    ("", 0),
    ("dvdf", 3)       # "vdf"
]

for s, expected in test_cases:
    result = length_of_longest_substring(s)
    print(f"Input: '{s}' -> Output: {result}, Expected: {expected}, Match: {result == expected}")

# Application: Find longest sequence of unique user IDs
user_sequence = "abcabcbb"
max_unique = length_of_longest_substring(user_sequence)
print(f"\nUser sequence: {user_sequence}")
print(f"Longest unique sequence length: {max_unique}")


## Question 16: Merge Intervals (Time Window Problem)

**Difficulty:** Medium  
**Topic:** Arrays, Sorting, Merging  
**Use Case:** Scheduling, time slot management, overlapping periods

Merge overlapping intervals.


In [ ]:
def merge_intervals(intervals):
    """
    Merge overlapping intervals.
    
    Time Complexity: O(n log n) due to sorting
    Space Complexity: O(n) for result
    
    Key Concept: Sort by start time, then merge overlapping intervals
    """
    if not intervals:
        return []
    
    # Sort by start time
    intervals.sort(key=lambda x: x[0])
    
    merged = [intervals[0]]
    
    for current in intervals[1:]:
        last = merged[-1]
        
        # If current overlaps with last, merge them
        if current[0] <= last[1]:
            merged[-1] = [last[0], max(last[1], current[1])]
        else:
            merged.append(current)
    
    return merged

# Test
print("Merge Intervals:")
print("=" * 60)
test_intervals = [[1, 3], [2, 6], [8, 10], [15, 18]]
result = merge_intervals(test_intervals)
print(f"Input: {test_intervals}")
print(f"Output: {result}")
print("Explanation: [1,3] and [2,6] overlap -> merge to [1,6]")

# Application: Merge overlapping booking time slots
booking_slots = [
    [9, 10],   # 9:00-10:00
    [9, 30, 10, 30],  # 9:30-10:30
    [11, 12],  # 11:00-12:00
    [11, 30, 12, 30]  # 11:30-12:30
]
merged_slots = merge_intervals(booking_slots)
print(f"\nBooking slots: {booking_slots}")
print(f"Merged slots: {merged_slots}")


## Question 17: Best Time to Buy and Sell Stock (Dynamic Programming)

**Difficulty:** Easy-Medium  
**Topic:** Dynamic Programming, Arrays  
**Use Case:** Optimization problems, finding maximum profit

Find maximum profit from buying and selling stock once.


In [ ]:
def max_profit(prices):
    """
    Find maximum profit from one buy and one sell.
    
    Time Complexity: O(n)
    Space Complexity: O(1)
    
    Key Concept: Track minimum price seen so far, calculate profit at each day
    """
    if not prices:
        return 0
    
    min_price = prices[0]
    max_profit = 0
    
    for price in prices[1:]:
        # Update minimum price seen so far
        min_price = min(min_price, price)
        # Calculate profit if we sell today
        max_profit = max(max_profit, price - min_price)
    
    return max_profit

def max_profit_multiple_transactions(prices):
    """
    Find maximum profit with multiple transactions allowed.
    
    Key Concept: Buy before every price increase, sell before every price decrease
    """
    if not prices:
        return 0
    
    profit = 0
    for i in range(1, len(prices)):
        if prices[i] > prices[i-1]:
            profit += prices[i] - prices[i-1]
    
    return profit

# Test
print("Best Time to Buy and Sell Stock:")
print("=" * 60)
prices = [7, 1, 5, 3, 6, 4]
result1 = max_profit(prices)
result2 = max_profit_multiple_transactions(prices)
print(f"Prices: {prices}")
print(f"Max profit (one transaction): {result1}")  # Buy at 1, sell at 6
print(f"Max profit (multiple transactions): {result2}")  # Buy at 1, sell at 5; buy at 3, sell at 6

# Application: Optimal pricing strategy
ride_prices = [10, 15, 12, 18, 20, 16]
optimal_profit = max_profit_multiple_transactions(ride_prices)
print(f"\nRide prices over time: {ride_prices}")
print(f"Maximum profit from dynamic pricing: {optimal_profit}")


## Question 18: Group Anagrams (Hash Maps)

**Difficulty:** Medium  
**Topic:** Hash Maps, String Manipulation  
**Use Case:** Grouping similar items, categorization

Group strings that are anagrams of each other.


In [ ]:
from collections import defaultdict

def group_anagrams(strs):
    """
    Group anagrams together.
    
    Time Complexity: O(n * k log k) where k is avg string length
    Space Complexity: O(n * k)
    
    Key Concept: Use sorted string as key to group anagrams
    """
    groups = defaultdict(list)
    
    for s in strs:
        # Sort characters to create key
        key = ''.join(sorted(s))
        groups[key].append(s)
    
    return list(groups.values())

# Alternative: Use character count as key (more efficient for long strings)
def group_anagrams_count(strs):
    """
    Use character frequency count as key.
    
    More efficient for longer strings.
    """
    groups = defaultdict(list)
    
    for s in strs:
        count = [0] * 26  # For lowercase letters
        for char in s:
            count[ord(char) - ord('a')] += 1
        key = tuple(count)
        groups[key].append(s)
    
    return list(groups.values())

# Test
print("Group Anagrams:")
print("=" * 60)
strs = ["eat", "tea", "tan", "ate", "nat", "bat"]
result = group_anagrams(strs)
print(f"Input: {strs}")
print(f"Output: {result}")
print("Explanation: 'eat', 'tea', 'ate' are anagrams; 'tan', 'nat' are anagrams")

# Application: Group similar trip patterns
trip_patterns = ["NYC-SF", "SF-NYC", "LA-CHI", "CHI-LA", "NYC-LA"]
# Normalize by sorting origin-destination
normalized = {}
for pattern in trip_patterns:
    cities = pattern.split('-')
    key = '-'.join(sorted(cities))
    if key not in normalized:
        normalized[key] = []
    normalized[key].append(pattern)
print(f"\nTrip patterns: {trip_patterns}")
print(f"Grouped by route: {list(normalized.values())}")


## Question 19: Implement LRU Cache (System Design Component)

**Difficulty:** Hard  
**Topic:** Hash Maps, Doubly Linked Lists  
**Use Case:** Caching, feature stores, memoization

Design and implement a Least Recently Used (LRU) cache.


In [ ]:
from collections import OrderedDict

class LRUCache:
    """
    LRU Cache implementation.
    
    Time Complexity: O(1) for both get and put
    Space Complexity: O(capacity)
    
    Key Concept: Use OrderedDict which maintains insertion order
    """
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = OrderedDict()
    
    def get(self, key):
        if key not in self.cache:
            return -1
        
        # Move to end (most recently used)
        self.cache.move_to_end(key)
        return self.cache[key]
    
    def put(self, key, value):
        if key in self.cache:
            # Update value and move to end
            self.cache[key] = value
            self.cache.move_to_end(key)
        else:
            if len(self.cache) >= self.capacity:
                # Remove least recently used (first item)
                self.cache.popitem(last=False)
            self.cache[key] = value

# Test
print("LRU Cache:")
print("=" * 60)
lru = LRUCache(2)
lru.put(1, "value1")
lru.put(2, "value2")
print(f"Get(1): {lru.get(1)}")  # Returns "value1"
lru.put(3, "value3")  # Evicts key 2
print(f"Get(2): {lru.get(2)}")  # Returns -1 (not found)
print(f"Get(3): {lru.get(3)}")  # Returns "value3"
print(f"Get(1): {lru.get(1)}")  # Returns "value1"

# Application: Cache frequently accessed features
feature_cache = LRUCache(capacity=100)
print("\nFeature cache example:")
feature_cache.put("user_123_features", {"rating": 4.5, "trips": 50})
feature_cache.put("user_456_features", {"rating": 4.8, "trips": 100})
print(f"Cached features for user_123: {feature_cache.get('user_123_features')}")


## Question 20: Find Peak Element (Binary Search Variant)

**Difficulty:** Medium  
**Topic:** Binary Search  
**Use Case:** Finding local maxima, optimization problems

Find a peak element (greater than neighbors) in an array.


In [ ]:
def find_peak_element(nums):
    """
    Find peak element using binary search.
    
    Time Complexity: O(log n)
    Space Complexity: O(1)
    
    Key Concept: If nums[mid] < nums[mid+1], peak is in right half
    """
    left, right = 0, len(nums) - 1
    
    while left < right:
        mid = left + (right - left) // 2
        
        if nums[mid] > nums[mid + 1]:
            # Peak is in left half (including mid)
            right = mid
        else:
            # Peak is in right half
            left = mid + 1
    
    return left

# Test
print("Find Peak Element:")
print("=" * 60)
nums = [1, 2, 3, 1]
peak_idx = find_peak_element(nums)
print(f"Array: {nums}")
print(f"Peak element at index: {peak_idx}, value: {nums[peak_idx]}")

# Application: Find peak demand hour
hourly_demand = [10, 15, 20, 25, 30, 28, 22, 18]
peak_hour = find_peak_element(hourly_demand)
print(f"\nHourly demand: {hourly_demand}")
print(f"Peak demand hour: {peak_hour} (value: {hourly_demand[peak_hour]})")


## Question 21: Word Break (Dynamic Programming)

**Difficulty:** Medium  
**Topic:** Dynamic Programming  
**Use Case:** Text processing, validation, parsing

Determine if a string can be segmented into dictionary words.


In [ ]:
def word_break(s, word_dict):
    """
    Check if string can be segmented into dictionary words.
    
    Time Complexity: O(n^2) where n is string length
    Space Complexity: O(n) for memoization
    
    Key Concept: DP - dp[i] = True if s[0:i] can be segmented
    """
    word_set = set(word_dict)
    n = len(s)
    dp = [False] * (n + 1)
    dp[0] = True  # Empty string is valid
    
    for i in range(1, n + 1):
        for j in range(i):
            # Check if s[0:j] is valid and s[j:i] is in dictionary
            if dp[j] and s[j:i] in word_set:
                dp[i] = True
                break
    
    return dp[n]

# Test
print("Word Break:")
print("=" * 60)
s = "leetcode"
word_dict = ["leet", "code"]
result = word_break(s, word_dict)
print(f"String: '{s}', Dictionary: {word_dict}")
print(f"Can be segmented: {result}")

# Application: Validate trip route code
route_codes = ["NYC", "SF", "LA", "CHI"]
route_string = "NYCSFLA"
is_valid_route = word_break(route_string, route_codes)
print(f"\nRoute string: '{route_string}', Valid codes: {route_codes}")
print(f"Is valid route: {is_valid_route}")


## Question 22: Implement Trie (Prefix Tree) - For Autocomplete

**Difficulty:** Medium  
**Topic:** Trie, Trees  
**Use Case:** Autocomplete, prefix matching, search suggestions

Implement a Trie data structure for efficient prefix searches.


In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end = False

class Trie:
    """
    Trie (Prefix Tree) implementation.
    
    Time Complexity:
    - Insert: O(m) where m is word length
    - Search: O(m)
    - StartsWith: O(m)
    
    Space Complexity: O(ALPHABET_SIZE * N * M) where N is number of words, M is avg length
    """
    
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end = True
    
    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return node.is_end
    
    def starts_with(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return False
            node = node.children[char]
        return True
    
    def get_all_with_prefix(self, prefix):
        """Get all words starting with prefix"""
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []
            node = node.children[char]
        
        # DFS to collect all words from this node
        results = []
        self._dfs(node, prefix, results)
        return results
    
    def _dfs(self, node, prefix, results):
        if node.is_end:
            results.append(prefix)
        for char, child in node.children.items():
            self._dfs(child, prefix + char, results)

# Test
print("Trie (Prefix Tree):")
print("=" * 60)
trie = Trie()
words = ["apple", "app", "application", "apply", "banana", "band"]
for word in words:
    trie.insert(word)

print(f"Inserted words: {words}")
print(f"Search 'app': {trie.search('app')}")
print(f"Search 'appl': {trie.search('appl')}")
print(f"Starts with 'app': {trie.starts_with('app')}")
print(f"All words with prefix 'app': {trie.get_all_with_prefix('app')}")

# Application: City name autocomplete
city_trie = Trie()
cities = ["New York", "Newark", "San Francisco", "San Diego", "Los Angeles"]
for city in cities:
    city_trie.insert(city.lower().replace(" ", ""))

print(f"\nCity autocomplete for 'new': {city_trie.get_all_with_prefix('new')}")
print(f"City autocomplete for 'san': {city_trie.get_all_with_prefix('san')}")


## Question 23: Calculate Distance Between Two Points (Geospatial)

**Difficulty:** Easy  
**Topic:** Math, Geospatial  
**Use Case:** Ride matching, distance calculations

Calculate distance between two geographic points using Haversine formula.


In [ ]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate great circle distance between two points on Earth.
    
    Uses Haversine formula for spherical distance calculation.
    
    Parameters:
    -----------
    lat1, lon1: float
        Latitude and longitude of first point (in degrees)
    lat2, lon2: float
        Latitude and longitude of second point (in degrees)
    
    Returns:
    --------
    float: Distance in kilometers
    """
    # Earth radius in kilometers
    R = 6371.0
    
    # Convert to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = (math.sin(dlat / 2)**2 + 
         math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c
    return distance

def euclidean_distance(x1, y1, x2, y2):
    """
    Euclidean distance (for 2D plane, not Earth surface).
    Useful for grid-based systems or small distances.
    """
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Test
print("Distance Calculations:")
print("=" * 60)
# NYC coordinates
nyc_lat, nyc_lon = 40.7128, -74.0060
# SF coordinates
sf_lat, sf_lon = 37.7749, -122.4194

distance_km = haversine_distance(nyc_lat, nyc_lon, sf_lat, sf_lon)
distance_miles = distance_km * 0.621371

print(f"NYC: ({nyc_lat}, {nyc_lon})")
print(f"SF: ({sf_lat}, {sf_lon})")
print(f"Distance: {distance_km:.2f} km ({distance_miles:.2f} miles)")

# Application: Find closest driver to rider
rider_location = (40.7580, -73.9855)  # Times Square, NYC
drivers = [
    {"id": 1, "location": (40.7614, -73.9776)},  # Central Park
    {"id": 2, "location": (40.7505, -73.9934)},  # Penn Station
    {"id": 3, "location": (40.7282, -73.9942)},  # SoHo
]

distances = []
for driver in drivers:
    dist = haversine_distance(
        rider_location[0], rider_location[1],
        driver["location"][0], driver["location"][1]
    )
    distances.append((driver["id"], dist))

closest_driver = min(distances, key=lambda x: x[1])
print(f"\nRider location: {rider_location}")
print(f"Closest driver: ID {closest_driver[0]}, distance: {closest_driver[1]:.2f} km")


## Question 24: Reservoir Sampling (Streaming Data)

**Difficulty:** Medium  
**Topic:** Probability, Algorithms  
**Use Case:** Sampling from large/streaming datasets

Implement reservoir sampling to randomly select k items from a stream of unknown size.


In [ ]:
import random

def reservoir_sampling(stream, k):
    """
    Randomly sample k items from a stream.
    
    Time Complexity: O(n) where n is stream size
    Space Complexity: O(k)
    
    Key Concept: Each item has k/n probability of being selected
    """
    reservoir = []
    
    for i, item in enumerate(stream):
        if i < k:
            # Fill reservoir with first k items
            reservoir.append(item)
        else:
            # Replace with probability k/(i+1)
            j = random.randint(0, i)
            if j < k:
                reservoir[j] = item
    
    return reservoir

# Test with stream
print("Reservoir Sampling:")
print("=" * 60)
stream = list(range(100))  # Stream of numbers 0-99
k = 10
sample = reservoir_sampling(stream, k)
print(f"Stream size: {len(stream)}, Sample size: {k}")
print(f"Sample: {sorted(sample)}")
print(f"Note: Each number has equal probability of being selected")

# Application: Sample trips for analysis without loading all data
def trip_stream_generator():
    """Generator simulating stream of trips"""
    for i in range(10000):
        yield {"trip_id": i, "distance": random.uniform(1, 50)}

print("\nSampling trips from stream:")
trip_sample = reservoir_sampling(trip_stream_generator(), 100)
print(f"Sampled {len(trip_sample)} trips out of 10,000")
print(f"Average distance in sample: {sum(t['distance'] for t in trip_sample) / len(trip_sample):.2f} km")


## Question 25: Calculate Percentiles and Quartiles (Statistics)

**Difficulty:** Easy-Medium  
**Topic:** Statistics, Arrays  
**Use Case:** Data analysis, outlier detection, reporting

Calculate percentiles and quartiles from a dataset.


In [ ]:
def calculate_percentile(data, percentile):
    """
    Calculate percentile value.
    
    Parameters:
    -----------
    data: List[float]
        Sorted data
    percentile: float
        Percentile (0-100)
    
    Returns:
    --------
    float: Percentile value
    """
    if not data:
        return None
    
    sorted_data = sorted(data)
    n = len(sorted_data)
    
    # Linear interpolation method
    index = (percentile / 100) * (n - 1)
    lower = int(index)
    upper = lower + 1
    
    if upper >= n:
        return sorted_data[-1]
    
    weight = index - lower
    return sorted_data[lower] * (1 - weight) + sorted_data[upper] * weight

def calculate_quartiles(data):
    """Calculate Q1, Q2 (median), Q3"""
    return {
        'Q1': calculate_percentile(data, 25),
        'Q2': calculate_percentile(data, 50),
        'Q3': calculate_percentile(data, 75)
    }

def detect_outliers_iqr(data):
    """
    Detect outliers using IQR method.
    
    Outliers: values < Q1 - 1.5*IQR or > Q3 + 1.5*IQR
    """
    quartiles = calculate_quartiles(data)
    iqr = quartiles['Q3'] - quartiles['Q1']
    
    lower_bound = quartiles['Q1'] - 1.5 * iqr
    upper_bound = quartiles['Q3'] + 1.5 * iqr
    
    outliers = [x for x in data if x < lower_bound or x > upper_bound]
    
    return outliers, lower_bound, upper_bound

# Test
print("Percentiles and Quartiles:")
print("=" * 60)
trip_durations = [15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 100, 120]  # Minutes

quartiles = calculate_quartiles(trip_durations)
print(f"Trip durations: {trip_durations}")
print(f"Q1 (25th percentile): {quartiles['Q1']:.2f}")
print(f"Q2 (Median, 50th percentile): {quartiles['Q2']:.2f}")
print(f"Q3 (75th percentile): {quartiles['Q3']:.2f}")

outliers, lower, upper = detect_outliers_iqr(trip_durations)
print(f"\nOutliers (IQR method): {outliers}")
print(f"Normal range: [{lower:.2f}, {upper:.2f}]")

# Application: Analyze wait times
wait_times = [2, 3, 3, 4, 5, 5, 6, 7, 8, 10, 15, 20]
p95 = calculate_percentile(wait_times, 95)
print(f"\nWait times: {wait_times}")
print(f"95th percentile wait time: {p95:.2f} minutes")
print(f"95% of rides have wait time <= {p95:.2f} minutes")


## Question 26: Implement Circular Buffer (Fixed-Size Queue)

**Difficulty:** Medium  
**Topic:** Arrays, Circular Buffer  
**Use Case:** Sliding windows, rate limiting, recent history tracking

Implement a circular buffer for efficient fixed-size queue operations.


In [ ]:
class CircularBuffer:
    """
    Circular Buffer implementation.
    
    Time Complexity: O(1) for all operations
    Space Complexity: O(capacity)
    
    Key Concept: Use modulo arithmetic to wrap around array indices
    """
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = [None] * capacity
        self.head = 0  # Write pointer
        self.size = 0  # Current number of elements
    
    def enqueue(self, value):
        """Add element to buffer"""
        self.buffer[self.head] = value
        self.head = (self.head + 1) % self.capacity
        
        if self.size < self.capacity:
            self.size += 1
    
    def get_all(self):
        """Get all elements in order (oldest to newest)"""
        if self.size == 0:
            return []
        
        result = []
        start = (self.head - self.size) % self.capacity
        
        for i in range(self.size):
            idx = (start + i) % self.capacity
            result.append(self.buffer[idx])
        
        return result
    
    def get_recent(self, n):
        """Get n most recent elements"""
        all_elements = self.get_all()
        return all_elements[-n:] if n <= len(all_elements) else all_elements
    
    def is_full(self):
        return self.size == self.capacity

# Test
print("Circular Buffer:")
print("=" * 60)
buffer = CircularBuffer(capacity=5)

# Add elements
for i in range(7):
    buffer.enqueue(i)
    print(f"After adding {i}: {buffer.get_all()}")

print(f"\nBuffer is full: {buffer.is_full()}")
print(f"Recent 3 elements: {buffer.get_recent(3)}")

# Application: Track recent ride requests
request_buffer = CircularBuffer(capacity=10)
requests = [f"request_{i}" for i in range(15)]
for req in requests:
    request_buffer.enqueue(req)

print(f"\nRecent 5 ride requests: {request_buffer.get_recent(5)}")


## Summary: Key Patterns and Strategies

### Common Patterns:

1. **Two Pointers**: Use start and end pointers moving towards each other
   - Examples: Two Sum, Container With Most Water

2. **Sliding Window**: Maintain window and slide it
   - Examples: Longest Substring, Maximum in Window

3. **Hash Map**: O(1) lookups and counting
   - Examples: Two Sum, Group Anagrams, Top K Frequent

4. **Binary Search**: Divide search space in half
   - Examples: Search in Sorted Array, Find Peak

5. **Dynamic Programming**: Solve subproblems, store results
   - Examples: Best Time to Buy/Sell, Word Break

6. **Stack/Queue**: LIFO/FIFO operations
   - Examples: Valid Parentheses, LRU Cache

7. **Trie**: Prefix tree for string operations
   - Examples: Autocomplete, Prefix Matching

### Problem-Solving Approach:

1. **Understand the problem**: Read carefully, identify constraints
2. **Brute force first**: Get working solution, then optimize
3. **Identify pattern**: Which data structure/algorithm fits?
4. **Optimize**: Improve time/space complexity
5. **Edge cases**: Handle empty inputs, single elements, etc.
6. **Test**: Verify with examples

### Time Complexity Cheat Sheet:

- O(1): Hash map lookup, array access
- O(log n): Binary search, balanced tree operations
- O(n): Single pass through array
- O(n log n): Sorting, divide and conquer
- O(n²): Nested loops
- O(2^n): Recursion without memoization

### Practice Recommendations:

1. Focus on arrays, hash maps, and sliding windows (most common)
2. Practice binary search variants
3. Understand when to use DP vs Greedy
4. Know common data structures (heap, deque, Trie)
5. Practice explaining your approach out loud

---


## Question 27: Reverse Linked List

**Difficulty:** Easy-Medium  
**Topic:** Linked Lists, Pointers  
**Use Case:** Data structure manipulation, reversing sequences

Reverse a singly linked list iteratively and recursively.


In [ ]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next
    
    def to_list(self):
        """Convert linked list to Python list"""
        result = []
        current = self
        while current:
            result.append(current.val)
            current = current.next
        return result
    
    @staticmethod
    def from_list(values):
        """Create linked list from Python list"""
        if not values:
            return None
        head = ListNode(values[0])
        current = head
        for val in values[1:]:
            current.next = ListNode(val)
            current = current.next
        return head

def reverse_linked_list_iterative(head):
    """
    Reverse linked list iteratively.
    
    Time Complexity: O(n)
    Space Complexity: O(1)
    
    Key Concept: Use three pointers to reverse links
    """
    prev = None
    current = head
    
    while current:
        next_node = current.next  # Store next node
        current.next = prev        # Reverse link
        prev = current             # Move prev forward
        current = next_node        # Move current forward
    
    return prev  # prev is new head

def reverse_linked_list_recursive(head):
    """
    Reverse linked list recursively.
    
    Time Complexity: O(n)
    Space Complexity: O(n) due to recursion stack
    
    Key Concept: Reverse rest of list, then reverse current link
    """
    if not head or not head.next:
        return head
    
    # Reverse rest of list
    new_head = reverse_linked_list_recursive(head.next)
    
    # Reverse current link
    head.next.next = head
    head.next = None
    
    return new_head

# Test
print("Reverse Linked List:")
print("=" * 60)
original = [1, 2, 3, 4, 5]
head1 = ListNode.from_list(original)
head2 = ListNode.from_list(original)

print(f"Original: {original}")
reversed_iter = reverse_linked_list_iterative(head1)
reversed_rec = reverse_linked_list_recursive(head2)

print(f"Iterative: {reversed_iter.to_list()}")
print(f"Recursive: {reversed_rec.to_list()}")


## Question 28: Validate Binary Search Tree

**Difficulty:** Medium  
**Topic:** Trees, Binary Search Tree  
**Use Case:** Tree validation, data structure integrity

Determine if a binary tree is a valid BST.


In [ ]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def is_valid_bst(root, min_val=float('-inf'), max_val=float('inf')):
    """
    Validate if binary tree is a valid BST.
    
    Time Complexity: O(n) where n is number of nodes
    Space Complexity: O(h) where h is height (due to recursion)
    
    Key Concept: Each node must be within valid range (min_val, max_val)
    """
    if not root:
        return True
    
    # Check if current node value is within valid range
    if root.val <= min_val or root.val >= max_val:
        return False
    
    # Recursively check left and right subtrees
    # Left subtree: all values must be < root.val
    # Right subtree: all values must be > root.val
    return (is_valid_bst(root.left, min_val, root.val) and
            is_valid_bst(root.right, root.val, max_val))

def is_valid_bst_inorder(root):
    """
    Alternative: Inorder traversal of valid BST gives sorted sequence.
    """
    def inorder(node):
        if not node:
            return []
        return inorder(node.left) + [node.val] + inorder(node.right)
    
    values = inorder(root)
    return values == sorted(values) and len(values) == len(set(values))

# Test
print("Validate Binary Search Tree:")
print("=" * 60)
# Valid BST
#       5
#      / \
#     3   7
#    / \
#   2   4
root1 = TreeNode(5)
root1.left = TreeNode(3)
root1.right = TreeNode(7)
root1.left.left = TreeNode(2)
root1.left.right = TreeNode(4)

# Invalid BST (4 > 3 but in left subtree of 5)
#       5
#      / \
#     3   7
#    / \
#   2   6  <- 6 > 5, invalid!
root2 = TreeNode(5)
root2.left = TreeNode(3)
root2.right = TreeNode(7)
root2.left.left = TreeNode(2)
root2.left.right = TreeNode(6)

print(f"Tree 1 (valid BST): {is_valid_bst(root1)}")
print(f"Tree 2 (invalid BST): {is_valid_bst(root2)}")


## Question 29: Maximum Depth of Binary Tree

**Difficulty:** Easy  
**Topic:** Trees, Recursion, DFS  
**Use Case:** Tree traversal, depth calculations

Find the maximum depth (height) of a binary tree.


In [ ]:
def max_depth_recursive(root):
    """
    Find maximum depth using recursion.
    
    Time Complexity: O(n)
    Space Complexity: O(h) where h is height
    """
    if not root:
        return 0
    
    return 1 + max(max_depth_recursive(root.left), 
                   max_depth_recursive(root.right))

def max_depth_iterative(root):
    """
    Find maximum depth using BFS (level-order traversal).
    
    Time Complexity: O(n)
    Space Complexity: O(w) where w is max width
    """
    if not root:
        return 0
    
    from collections import deque
    queue = deque([root])
    depth = 0
    
    while queue:
        depth += 1
        level_size = len(queue)
        
        for _ in range(level_size):
            node = queue.popleft()
            if node.left:
                queue.append(node.left)
            if node.right:
                queue.append(node.right)
    
    return depth

# Test
print("Maximum Depth of Binary Tree:")
print("=" * 60)
#     3
#    / \
#   9   20
#      /  \
#     15   7
root = TreeNode(3)
root.left = TreeNode(9)
root.right = TreeNode(20)
root.right.left = TreeNode(15)
root.right.right = TreeNode(7)

print(f"Max depth (recursive): {max_depth_recursive(root)}")
print(f"Max depth (iterative): {max_depth_iterative(root)}")


## Question 30: Graph - BFS and DFS Traversal

**Difficulty:** Medium  
**Topic:** Graphs, BFS, DFS  
**Use Case:** Graph exploration, path finding, connected components

Implement BFS and DFS for graph traversal.


In [ ]:
from collections import deque, defaultdict

class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
    
    def add_edge(self, u, v):
        self.graph[u].append(v)
        # For undirected graph, also add reverse
        # self.graph[v].append(u)
    
    def bfs(self, start):
        """
        Breadth-First Search.
        
        Time Complexity: O(V + E)
        Space Complexity: O(V)
        
        Key Concept: Use queue, visit neighbors level by level
        """
        visited = set()
        queue = deque([start])
        result = []
        
        while queue:
            node = queue.popleft()
            if node not in visited:
                visited.add(node)
                result.append(node)
                # Add all neighbors to queue
                for neighbor in self.graph[node]:
                    if neighbor not in visited:
                        queue.append(neighbor)
        
        return result
    
    def dfs_recursive(self, start, visited=None, result=None):
        """
        Depth-First Search (recursive).
        
        Time Complexity: O(V + E)
        Space Complexity: O(V) for recursion stack
        """
        if visited is None:
            visited = set()
        if result is None:
            result = []
        
        visited.add(start)
        result.append(start)
        
        for neighbor in self.graph[start]:
            if neighbor not in visited:
                self.dfs_recursive(neighbor, visited, result)
        
        return result
    
    def dfs_iterative(self, start):
        """
        Depth-First Search (iterative using stack).
        
        Time Complexity: O(V + E)
        Space Complexity: O(V)
        """
        visited = set()
        stack = [start]
        result = []
        
        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                result.append(node)
                # Add neighbors in reverse order to maintain left-to-right
                for neighbor in reversed(self.graph[node]):
                    if neighbor not in visited:
                        stack.append(neighbor)
        
        return result

# Test
print("Graph Traversal (BFS and DFS):")
print("=" * 60)
g = Graph()
g.add_edge(0, 1)
g.add_edge(0, 2)
g.add_edge(1, 2)
g.add_edge(2, 0)
g.add_edge(2, 3)
g.add_edge(3, 3)

print("Graph structure:")
print("0 -> 1, 2")
print("1 -> 2")
print("2 -> 0, 3")
print("3 -> 3")

print(f"\nBFS starting from 2: {g.bfs(2)}")
print(f"DFS (recursive) starting from 2: {g.dfs_recursive(2)}")
print(f"DFS (iterative) starting from 2: {g.dfs_iterative(2)}")


## Question 31: Climbing Stairs (Fibonacci Variant)

**Difficulty:** Easy  
**Topic:** Dynamic Programming, Memoization  
**Use Case:** Optimization problems, counting combinations

Find number of ways to climb n stairs (can take 1 or 2 steps at a time).


In [ ]:
def climb_stairs_recursive(n):
    """
    Recursive solution (inefficient, O(2^n)).
    """
    if n <= 2:
        return n
    return climb_stairs_recursive(n - 1) + climb_stairs_recursive(n - 2)

def climb_stairs_memoization(n, memo={}):
    """
    Recursive with memoization.
    
    Time Complexity: O(n)
    Space Complexity: O(n)
    """
    if n in memo:
        return memo[n]
    if n <= 2:
        return n
    memo[n] = climb_stairs_memoization(n - 1, memo) + climb_stairs_memoization(n - 2, memo)
    return memo[n]

def climb_stairs_dp(n):
    """
    Dynamic programming (bottom-up).
    
    Time Complexity: O(n)
    Space Complexity: O(1)
    
    Key Concept: Only need last two values (like Fibonacci)
    """
    if n <= 2:
        return n
    
    prev2 = 1  # ways to reach step 1
    prev1 = 2  # ways to reach step 2
    
    for i in range(3, n + 1):
        current = prev1 + prev2
        prev2 = prev1
        prev1 = current
    
    return prev1

# Test
print("Climbing Stairs:")
print("=" * 60)
for n in [1, 2, 3, 4, 5, 10]:
    result = climb_stairs_dp(n)
    print(f"n={n}: {result} ways")
    # Verification: For n stairs, result should be (n+1)th Fibonacci number

print("\nPattern: 1, 2, 3, 5, 8, 13, 21, ... (Fibonacci sequence)")
print("Reason: Ways(n) = Ways(n-1) + Ways(n-2)")

# Application: Number of combinations for multi-step processes
steps = 5
print(f"\nWays to reach {steps} steps: {climb_stairs_dp(steps)}")


## Question 32: Python Fundamentals - List Comprehensions and Generators

**Difficulty:** Easy  
**Topic:** Python Fundamentals  
**Use Case:** Efficient data processing, memory optimization

Demonstrate list comprehensions, generator expressions, and their differences.


In [ ]:
# List Comprehension: Creates list immediately (eager evaluation)
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
squared_list = [x**2 for x in numbers if x % 2 == 0]
print("List Comprehension:")
print(f"Squared even numbers: {squared_list}")

# Generator Expression: Lazy evaluation (saves memory)
squared_gen = (x**2 for x in numbers if x % 2 == 0)
print("\nGenerator Expression:")
print(f"Type: {type(squared_gen)}")
print(f"Values: {list(squared_gen)}")  # Convert to list to see values

# Generator Function: More complex logic
def fibonacci_generator(n):
    """Generate first n Fibonacci numbers"""
    a, b = 0, 1
    count = 0
    while count < n:
        yield a
        a, b = b, a + b
        count += 1

print("\nGenerator Function (Fibonacci):")
fib_gen = fibonacci_generator(10)
print(f"First 10 Fibonacci: {list(fib_gen)}")

# Memory comparison
print("\nMemory Comparison:")
import sys

# List comprehension - all in memory
big_list = [x**2 for x in range(1000000)]
print(f"List size: {sys.getsizeof(big_list)} bytes")

# Generator - lazy evaluation
big_gen = (x**2 for x in range(1000000))
print(f"Generator size: {sys.getsizeof(big_gen)} bytes")

# Application: Process large datasets efficiently
def process_trips_generator(trip_ids):
    """Generator to process trips one at a time (memory efficient)"""
    for trip_id in trip_ids:
        # Simulate processing
        yield {"trip_id": trip_id, "processed": True}

trip_ids = range(1000000)
# Using generator: processes one at a time
processed = process_trips_generator(trip_ids)
print(f"\nProcessed trips (first 5): {[next(processed) for _ in range(5)]}")


## Question 33: Python Decorators

**Difficulty:** Medium  
**Topic:** Python Fundamentals, Decorators  
**Use Case:** Function wrapping, caching, logging, timing

Implement decorators for timing, caching, and logging.


In [ ]:
import time
from functools import wraps, lru_cache

# Timing Decorator
def timer(func):
    """Decorator to measure function execution time"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

# Caching Decorator (simple version)
def cache_decorator(func):
    """Simple memoization decorator"""
    cache = {}
    @wraps(func)
    def wrapper(*args):
        if args in cache:
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result
    return wrapper

# Logging Decorator
def log_calls(func):
    """Decorator to log function calls"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned {result}")
        return result
    return wrapper

# Using decorators
@timer
@cache_decorator
def fibonacci_slow(n):
    """Slow Fibonacci without memoization"""
    if n <= 1:
        return n
    return fibonacci_slow(n - 1) + fibonacci_slow(n - 2)

@timer
@lru_cache(maxsize=128)  # Built-in caching decorator
def fibonacci_fast(n):
    """Fast Fibonacci with lru_cache"""
    if n <= 1:
        return n
    return fibonacci_fast(n - 1) + fibonacci_fast(n - 2)

@log_calls
def calculate_revenue(price, quantity):
    return price * quantity

# Test
print("Decorators:")
print("=" * 60)
print("1. Timing and Caching:")
print("Slow version (with custom cache):")
fib_slow_result = fibonacci_slow(30)
print("\nFast version (with lru_cache):")
fib_fast_result = fibonacci_fast(30)

print("\n2. Logging:")
calculate_revenue(10, 5)


## Question 34: Missing Number in Array

**Difficulty:** Easy  
**Topic:** Arrays, Math  
**Use Case:** Data validation, finding gaps

Find the missing number in an array containing n distinct numbers from 0 to n.


In [ ]:
def missing_number_sum(nums):
    """
    Using sum formula.
    
    Time Complexity: O(n)
    Space Complexity: O(1)
    
    Key Concept: Sum of 0 to n = n*(n+1)/2
    """
    n = len(nums)
    expected_sum = n * (n + 1) // 2
    actual_sum = sum(nums)
    return expected_sum - actual_sum

def missing_number_xor(nums):
    """
    Using XOR properties.
    
    Time Complexity: O(n)
    Space Complexity: O(1)
    
    Key Concept: a XOR a = 0, a XOR 0 = a
    """
    n = len(nums)
    missing = n
    for i, num in enumerate(nums):
        missing ^= i ^ num
    return missing

def missing_number_set(nums):
    """
    Using set for lookup.
    
    Time Complexity: O(n)
    Space Complexity: O(n)
    """
    num_set = set(nums)
    n = len(nums)
    for i in range(n + 1):
        if i not in num_set:
            return i
    return -1

# Test
print("Missing Number:")
print("=" * 60)
test_cases = [
    [3, 0, 1],  # Missing 2
    [0, 1],     # Missing 2
    [9, 6, 4, 2, 3, 5, 7, 0, 1]  # Missing 8
]

for nums in test_cases:
    result1 = missing_number_sum(nums)
    result2 = missing_number_xor(nums)
    result3 = missing_number_set(nums)
    print(f"Array: {nums}")
    print(f"Missing (sum): {result1}, (XOR): {result2}, (set): {result3}")
    print("-" * 60)


## Question 35: Rotate Array

**Difficulty:** Medium  
**Topic:** Arrays, Two Pointers  
**Use Case:** Array manipulation, circular shifts

Rotate an array to the right by k steps.


In [ ]:
def rotate_array_slicing(nums, k):
    """
    Using array slicing.
    
    Time Complexity: O(n)
    Space Complexity: O(n)
    """
    n = len(nums)
    k = k % n  # Handle k > n
    nums[:] = nums[-k:] + nums[:-k]
    return nums

def rotate_array_reverse(nums, k):
    """
    Using reverse trick (in-place, O(1) space).
    
    Time Complexity: O(n)
    Space Complexity: O(1)
    
    Key Concept: Reverse entire array, then reverse first k and last n-k
    """
    n = len(nums)
    k = k % n
    
    # Reverse entire array
    nums.reverse()
    
    # Reverse first k elements
    nums[:k] = reversed(nums[:k])
    
    # Reverse remaining elements
    nums[k:] = reversed(nums[k:])
    
    return nums

def rotate_array_cyclic(nums, k):
    """
    Cyclic replacement (in-place, O(1) space).
    
    Time Complexity: O(n)
    Space Complexity: O(1)
    """
    n = len(nums)
    k = k % n
    start = count = 0
    
    while count < n:
        current, prev = start, nums[start]
        while True:
            next_idx = (current + k) % n
            nums[next_idx], prev = prev, nums[next_idx]
            current = next_idx
            count += 1
            if start == current:
                break
        start += 1
    
    return nums

# Test
print("Rotate Array:")
print("=" * 60)
test_array = [1, 2, 3, 4, 5, 6, 7]
k = 3
print(f"Original: {test_array}, k={k}")

# Method 1: Slicing
arr1 = test_array.copy()
result1 = rotate_array_slicing(arr1, k)
print(f"Slicing method: {result1}")

# Method 2: Reverse
arr2 = test_array.copy()
result2 = rotate_array_reverse(arr2, k)
print(f"Reverse method: {result2}")

# Method 3: Cyclic
arr3 = test_array.copy()
result3 = rotate_array_cyclic(arr3, k)
print(f"Cyclic method: {result3}")

print(f"\nExpected: [5, 6, 7, 1, 2, 3, 4]")


## Question 36: Container With Most Water

**Difficulty:** Medium  
**Topic:** Two Pointers, Greedy  
**Use Case:** Optimization problems, area calculations

Find two lines that together with x-axis forms a container that holds the most water.


In [ ]:
def max_area(height):
    """
    Two-pointer approach.
    
    Time Complexity: O(n)
    Space Complexity: O(1)
    
    Key Concept: Start with widest container, move pointer with smaller height
    """
    left, right = 0, len(height) - 1
    max_water = 0
    
    while left < right:
        # Calculate area
        width = right - left
        current_area = width * min(height[left], height[right])
        max_water = max(max_water, current_area)
        
        # Move pointer with smaller height
        if height[left] < height[right]:
            left += 1
        else:
            right -= 1
    
    return max_water

# Test
print("Container With Most Water:")
print("=" * 60)
heights = [1, 8, 6, 2, 5, 4, 8, 3, 7]
result = max_area(heights)
print(f"Heights: {heights}")
print(f"Maximum water: {result}")
print("Explanation: Lines at index 1 (height 8) and index 8 (height 7)")
print(f"Area = (8-1) * min(8, 7) = 7 * 7 = {result}")


## Question 37: String to Integer (atoi)

**Difficulty:** Medium  
**Topic:** String Manipulation  
**Use Case:** Parsing, input validation

Implement atoi function that converts string to integer.


In [ ]:
def my_atoi(s):
    """
    Convert string to integer.
    
    Handle:
    - Leading whitespace
    - Optional +/- sign
    - Read digits until non-digit or end
    - Clamp to [-2^31, 2^31 - 1]
    
    Time Complexity: O(n)
    Space Complexity: O(1)
    """
    if not s:
        return 0
    
    # Remove leading whitespace
    s = s.strip()
    if not s:
        return 0
    
    # Determine sign
    sign = 1
    if s[0] == '-':
        sign = -1
        s = s[1:]
    elif s[0] == '+':
        s = s[1:]
    
    # Read digits
    result = 0
    for char in s:
        if not char.isdigit():
            break
        result = result * 10 + int(char)
        
        # Check overflow
        if sign == 1 and result > 2**31 - 1:
            return 2**31 - 1
        if sign == -1 and result > 2**31:
            return -2**31
    
    return sign * result

# Test
print("String to Integer (atoi):")
print("=" * 60)
test_cases = [
    "42",
    "   -42",
    "4193 with words",
    "words and 987",
    "-91283472332",  # Overflow
    "91283472332",   # Overflow
]

for s in test_cases:
    result = my_atoi(s)
    print(f"'{s}' -> {result}")


## Question 38: Count and Say Sequence

**Difficulty:** Medium  
**Topic:** Strings, Recursion  
**Use Case:** Sequence generation, pattern recognition

Generate the nth term of the count-and-say sequence.


In [ ]:
def count_and_say(n):
    """
    Generate count-and-say sequence.
    
    Sequence: "1", "11", "21", "1211", "111221", ...
    
    Time Complexity: O(2^n) approximately
    Space Complexity: O(2^n)
    """
    if n == 1:
        return "1"
    
    prev = count_and_say(n - 1)
    result = []
    count = 1
    current_char = prev[0]
    
    for i in range(1, len(prev)):
        if prev[i] == current_char:
            count += 1
        else:
            result.append(str(count) + current_char)
            count = 1
            current_char = prev[i]
    
    result.append(str(count) + current_char)
    return ''.join(result)

# Iterative version (more efficient)
def count_and_say_iterative(n):
    result = "1"
    for _ in range(n - 1):
        new_result = []
        count = 1
        current_char = result[0]
        
        for i in range(1, len(result)):
            if result[i] == current_char:
                count += 1
            else:
                new_result.append(str(count) + current_char)
                count = 1
                current_char = result[i]
        
        new_result.append(str(count) + current_char)
        result = ''.join(new_result)
    
    return result

# Test
print("Count and Say:")
print("=" * 60)
for n in range(1, 8):
    result = count_and_say_iterative(n)
    print(f"n={n}: {result}")


## Question 39: Permutations (Backtracking)

**Difficulty:** Medium  
**Topic:** Backtracking, Recursion  
**Use Case:** Generating combinations, permutations, search problems

Generate all permutations of an array.


In [ ]:
def permute_backtrack(nums):
    """
    Generate all permutations using backtracking.
    
    Time Complexity: O(n! * n)
    Space Complexity: O(n) for recursion stack
    
    Key Concept: Swap elements, recurse, then swap back
    """
    result = []
    
    def backtrack(start):
        if start == len(nums):
            result.append(nums[:])  # Make a copy
            return
        
        for i in range(start, len(nums)):
            # Swap
            nums[start], nums[i] = nums[i], nums[start]
            # Recurse
            backtrack(start + 1)
            # Backtrack (swap back)
            nums[start], nums[i] = nums[i], nums[start]
    
    backtrack(0)
    return result

def permute_visited(nums):
    """
    Alternative: Using visited array.
    """
    result = []
    visited = [False] * len(nums)
    
    def backtrack(current_perm):
        if len(current_perm) == len(nums):
            result.append(current_perm[:])
            return
        
        for i in range(len(nums)):
            if not visited[i]:
                visited[i] = True
                current_perm.append(nums[i])
                backtrack(current_perm)
                current_perm.pop()
                visited[i] = False
    
    backtrack([])
    return result

# Test
print("Permutations:")
print("=" * 60)
nums = [1, 2, 3]
result1 = permute_backtrack(nums.copy())
result2 = permute_visited(nums.copy())
print(f"Input: {nums}")
print(f"All permutations ({len(result1)} total):")
for perm in result1:
    print(f"  {perm}")

# Using itertools (Python built-in)
from itertools import permutations
print(f"\nUsing itertools.permutations: {list(permutations(nums))}")


## Question 40: Python Context Managers (with statement)

**Difficulty:** Medium  
**Topic:** Python Fundamentals  
**Use Case:** Resource management, file operations, database connections

Implement context managers using class and decorator approaches.


In [ ]:
from contextlib import contextmanager

# Context Manager using class
class FileManager:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None
    
    def __enter__(self):
        print(f"Opening file {self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Closing file {self.filename}")
        if self.file:
            self.file.close()
        return False  # Don't suppress exceptions

# Context Manager using decorator
@contextmanager
def file_manager(filename, mode):
    """Context manager using @contextmanager decorator"""
    print(f"Opening file {filename}")
    file = open(filename, mode)
    try:
        yield file
    finally:
        print(f"Closing file {filename}")
        file.close()

# Custom context manager for timing
class TimerContext:
    def __enter__(self):
        self.start = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.elapsed = time.time() - self.start
        print(f"Elapsed time: {self.elapsed:.4f} seconds")
        return False

# Test
print("Context Managers:")
print("=" * 60)

# Using class-based context manager
with TimerContext():
    # Simulate some work
    sum([i**2 for i in range(1000000)])

# Custom context manager example
print("\n1. Class-based context manager:")
class DatabaseConnection:
    def __enter__(self):
        print("Connecting to database...")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Closing database connection...")
        return False
    
    def query(self, sql):
        print(f"Executing: {sql}")

with DatabaseConnection() as db:
    db.query("SELECT * FROM trips")

# Application: Resource management for model inference
print("\n2. Model inference context manager:")
class ModelInference:
    def __init__(self, model_name):
        self.model_name = model_name
        self.model = None
    
    def __enter__(self):
        print(f"Loading model {self.model_name}...")
        # Simulate model loading
        self.model = f"loaded_{self.model_name}"
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Unloading model {self.model_name}...")
        self.model = None
        return False
    
    def predict(self, data):
        return f"prediction for {data}"

with ModelInference("cancellation_model") as model:
    result = model.predict("trip_data")
    print(result)


## Question 41: Longest Common Prefix

**Difficulty:** Easy  
**Topic:** Strings  
**Use Case:** String matching, prefix operations

Find the longest common prefix string amongst an array of strings.


In [ ]:
def longest_common_prefix(strs):
    """
    Find longest common prefix.
    
    Time Complexity: O(S) where S is sum of all characters
    Space Complexity: O(1)
    
    Key Concept: Compare character by character
    """
    if not strs:
        return ""
    
    # Use first string as reference
    prefix = strs[0]
    
    for i in range(1, len(strs)):
        # Compare with current string
        while not strs[i].startswith(prefix):
            prefix = prefix[:-1]
            if not prefix:
                return ""
    
    return prefix

def longest_common_prefix_vertical(strs):
    """
    Vertical scanning: compare character by character.
    """
    if not strs:
        return ""
    
    for i in range(len(strs[0])):
        char = strs[0][i]
        for j in range(1, len(strs)):
            if i == len(strs[j]) or strs[j][i] != char:
                return strs[0][:i]
    
    return strs[0]

# Test
print("Longest Common Prefix:")
print("=" * 60)
test_cases = [
    ["flower", "flow", "flight"],
    ["dog", "racecar", "car"],
    ["interspecies", "interstellar", "interstate"],
    ["throne", "throne"],
    ["", "b"],
]

for strs in test_cases:
    result1 = longest_common_prefix(strs.copy())
    result2 = longest_common_prefix_vertical(strs.copy())
    print(f"Input: {strs}")
    print(f"Prefix (method 1): '{result1}', (method 2): '{result2}'")
    print("-" * 60)


## Question 42: First Unique Character in String

**Difficulty:** Easy  
**Topic:** Hash Maps, Strings  
**Use Case:** Finding first occurrence, character frequency

Find the first non-repeating character in a string.


In [ ]:
def first_uniq_char(s):
    """
    Find first unique character.
    
    Time Complexity: O(n)
    Space Complexity: O(1) - at most 26 characters
    
    Key Concept: Count frequencies, then find first with count=1
    """
    from collections import Counter
    count = Counter(s)
    
    for i, char in enumerate(s):
        if count[char] == 1:
            return i
    
    return -1

def first_uniq_char_dict(s):
    """Alternative using dict"""
    char_count = {}
    
    # Count frequencies
    for char in s:
        char_count[char] = char_count.get(char, 0) + 1
    
    # Find first unique
    for i, char in enumerate(s):
        if char_count[char] == 1:
            return i
    
    return -1

# Test
print("First Unique Character:")
print("=" * 60)
test_cases = [
    "leetcode",    # 'l' at index 0
    "loveleetcode", # 'v' at index 2
    "aabb",        # -1
    "abcdef",      # 'a' at index 0
]

for s in test_cases:
    result = first_uniq_char(s)
    if result != -1:
        print(f"'{s}': First unique char is '{s[result]}' at index {result}")
    else:
        print(f"'{s}': No unique character (return {result})")


## Question 43: Reverse Words in String

**Difficulty:** Medium  
**Topic:** Strings, Two Pointers  
**Use Case:** Text processing, string manipulation

Reverse the order of words in a string.


In [ ]:
def reverse_words_split(s):
    """
    Using split and join.
    
    Time Complexity: O(n)
    Space Complexity: O(n)
    """
    words = s.split()
    return ' '.join(reversed(words))

def reverse_words_inplace(s):
    """
    Reverse words in-place using two pointers.
    
    Time Complexity: O(n)
    Space Complexity: O(1) if we ignore output string
    """
    # Convert to list (strings are immutable)
    chars = list(s)
    n = len(chars)
    
    # Reverse entire string
    def reverse(l, r):
        while l < r:
            chars[l], chars[r] = chars[r], chars[l]
            l += 1
            r -= 1
    
    # Reverse entire string first
    reverse(0, n - 1)
    
    # Reverse each word
    start = 0
    for i in range(n + 1):
        if i == n or chars[i] == ' ':
            reverse(start, i - 1)
            start = i + 1
    
    return ''.join(chars).strip()

# Test
print("Reverse Words in String:")
print("=" * 60)
test_cases = [
    "the sky is blue",
    "  hello world  ",
    "a good   example",
    "  Bob    Loves  Alice   ",
]

for s in test_cases:
    result1 = reverse_words_split(s)
    result2 = reverse_words_inplace(s)
    print(f"Input: '{s}'")
    print(f"Output (split): '{result1}'")
    print(f"Output (inplace): '{result2}'")
    print("-" * 60)


## Question 44: Implement Queue using Stacks

**Difficulty:** Easy-Medium  
**Topic:** Stacks, Queues  
**Use Case:** Data structure implementation, understanding LIFO/FIFO

Implement a queue using two stacks.


In [ ]:
class MyQueue:
    """
    Queue using two stacks.
    
    Time Complexity:
    - push: O(1)
    - pop: O(n) amortized
    - peek: O(n) amortized
    
    Space Complexity: O(n)
    
    Key Concept: Use two stacks - one for enqueue, one for dequeue
    """
    
    def __init__(self):
        self.stack1 = []  # For push
        self.stack2 = []  # For pop
    
    def push(self, x):
        """Enqueue"""
        self.stack1.append(x)
    
    def pop(self):
        """Dequeue"""
        self.peek()  # Move elements if needed
        return self.stack2.pop()
    
    def peek(self):
        """Front element"""
        if not self.stack2:
            # Move all elements from stack1 to stack2
            while self.stack1:
                self.stack2.append(self.stack1.pop())
        return self.stack2[-1]
    
    def empty(self):
        return len(self.stack1) == 0 and len(self.stack2) == 0

# Test
print("Queue using Stacks:")
print("=" * 60)
queue = MyQueue()
queue.push(1)
queue.push(2)
queue.push(3)

print(f"Peek: {queue.peek()}")  # Should be 1
print(f"Pop: {queue.pop()}")    # Should be 1
print(f"Peek: {queue.peek()}")  # Should be 2
print(f"Empty: {queue.empty()}")  # False
queue.pop()
queue.pop()
print(f"Empty after popping all: {queue.empty()}")  # True


## Question 45: Python Iterators and Iterables

**Difficulty:** Medium  
**Topic:** Python Fundamentals  
**Use Case:** Custom iteration, lazy evaluation

Implement custom iterators and understand iterables.


In [ ]:
# Custom Iterator Class
class CountDown:
    """Iterator that counts down from start to stop"""
    
    def __init__(self, start, stop=0):
        self.current = start
        self.stop = stop
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current < self.stop:
            raise StopIteration
        else:
            self.current -= 1
            return self.current + 1

# Custom Iterable (uses generator)
class Squares:
    """Iterable that yields squares of numbers"""
    
    def __init__(self, n):
        self.n = n
    
    def __iter__(self):
        for i in range(self.n):
            yield i ** 2

# Iterator using generator function
def fibonacci_iterator(n):
    """Generator function for Fibonacci"""
    a, b = 0, 1
    count = 0
    while count < n:
        yield a
        a, b = b, a + b
        count += 1

# Test
print("Iterators and Iterables:")
print("=" * 60)

print("1. CountDown Iterator:")
countdown = CountDown(5, 0)
for num in countdown:
    print(f"  {num}", end=" ")
print()

print("\n2. Squares Iterable:")
squares = Squares(5)
for square in squares:
    print(f"  {square}", end=" ")
print()

print("\n3. Fibonacci Generator:")
fib = fibonacci_iterator(10)
print(f"  First 10 Fibonacci: {list(fib)}")

# Application: Custom iterator for trip data
class TripIterator:
    """Iterator for processing trips"""
    
    def __init__(self, trip_ids):
        self.trip_ids = trip_ids
        self.index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= len(self.trip_ids):
            raise StopIteration
        trip_id = self.trip_ids[self.index]
        self.index += 1
        return {"trip_id": trip_id, "status": "processed"}

print("\n4. Custom Trip Iterator:")
trips = TripIterator([101, 102, 103, 104])
for trip in trips:
    print(f"  {trip}")


## Question 46: Python Lambda and Higher-Order Functions

**Difficulty:** Easy  
**Topic:** Python Fundamentals  
**Use Case:** Functional programming, data transformation

Demonstrate lambda functions, map, filter, and reduce.


In [ ]:
from functools import reduce

# Lambda functions
square = lambda x: x ** 2
add = lambda x, y: x + y

print("Lambda Functions:")
print("=" * 60)
print(f"square(5) = {square(5)}")
print(f"add(3, 4) = {add(3, 4)}")

# Map: Apply function to each element
numbers = [1, 2, 3, 4, 5]
squared = list(map(lambda x: x**2, numbers))
print(f"\nMap: {numbers} -> {squared}")

# Filter: Filter elements based on condition
even_numbers = list(filter(lambda x: x % 2 == 0, numbers))
print(f"Filter (even): {numbers} -> {even_numbers}")

# Reduce: Reduce sequence to single value
sum_all = reduce(lambda x, y: x + y, numbers)
product = reduce(lambda x, y: x * y, numbers)
print(f"Reduce (sum): {numbers} -> {sum_all}")
print(f"Reduce (product): {numbers} -> {product}")

# Combined: Map + Filter
even_squares = list(map(lambda x: x**2, filter(lambda x: x % 2 == 0, numbers)))
print(f"Even squares: {even_squares}")

# Application: Data processing
trips = [
    {"distance": 5.2, "fare": 12.50},
    {"distance": 3.1, "fare": 8.00},
    {"distance": 8.7, "fare": 20.00},
    {"distance": 2.5, "fare": 6.50},
]

# Filter long trips (>5km)
long_trips = list(filter(lambda t: t["distance"] > 5, trips))
print(f"\nLong trips (>5km): {[t['distance'] for t in long_trips]}")

# Map: Calculate fare per km
fare_per_km = list(map(lambda t: t["fare"] / t["distance"], trips))
print(f"Fare per km: {[f'{f:.2f}' for f in fare_per_km]}")

# Reduce: Total revenue
total_revenue = reduce(lambda x, y: x + y, map(lambda t: t["fare"], trips))
print(f"Total revenue: ${total_revenue:.2f}")


## Question 47: Find All Anagrams in String

**Difficulty:** Medium  
**Topic:** Sliding Window, Hash Maps  
**Use Case:** Pattern matching, substring search

Find all start indices of anagrams of pattern in string.


In [ ]:
def find_anagrams(s, p):
    """
    Find all anagram indices using sliding window.
    
    Time Complexity: O(n) where n is len(s)
    Space Complexity: O(k) where k is len(p)
    
    Key Concept: Use sliding window with character frequency map
    """
    from collections import Counter
    
    result = []
    p_count = Counter(p)
    window_count = Counter()
    
    # Window size
    window_size = len(p)
    
    for i in range(len(s)):
        # Add current character
        window_count[s[i]] += 1
        
        # Remove character leaving window
        if i >= window_size:
            window_count[s[i - window_size]] -= 1
            if window_count[s[i - window_size]] == 0:
                del window_count[s[i - window_size]]
        
        # Check if window matches pattern
        if i >= window_size - 1 and window_count == p_count:
            result.append(i - window_size + 1)
    
    return result

# Test
print("Find All Anagrams:")
print("=" * 60)
s = "cbaebabacd"
p = "abc"
result = find_anagrams(s, p)
print(f"String: '{s}', Pattern: '{p}'")
print(f"Anagram indices: {result}")
print(f"Anagrams found: {[s[i:i+len(p)] for i in result]}")

# Another example
s2 = "abab"
p2 = "ab"
result2 = find_anagrams(s2, p2)
print(f"\nString: '{s2}', Pattern: '{p2}'")
print(f"Anagram indices: {result2}")


## Question 48: Python Class Methods, Static Methods, and Properties

**Difficulty:** Easy-Medium  
**Topic:** Python OOP  
**Use Case:** Object-oriented design, encapsulation

Understand @classmethod, @staticmethod, and @property decorators.


In [ ]:
class Trip:
    """Demonstrate instance methods, class methods, static methods, and properties"""
    
    # Class variable
    total_trips = 0
    base_fare = 2.50
    
    def __init__(self, distance, multiplier=1.0):
        self.distance = distance
        self.multiplier = multiplier
        Trip.total_trips += 1
        self._fare = None  # Private attribute
    
    # Instance method: operates on instance
    def calculate_fare(self):
        return self.distance * Trip.base_fare * self.multiplier
    
    # Property: accessed like attribute but computed
    @property
    def fare(self):
        if self._fare is None:
            self._fare = self.calculate_fare()
        return self._fare
    
    @fare.setter
    def fare(self, value):
        if value < 0:
            raise ValueError("Fare cannot be negative")
        self._fare = value
    
    # Class method: operates on class, receives class as first arg
    @classmethod
    def from_miles(cls, miles):
        """Alternative constructor: create Trip from miles"""
        km = miles * 1.60934
        return cls(km)
    
    @classmethod
    def set_base_fare(cls, fare):
        """Class method to change base fare for all trips"""
        cls.base_fare = fare
    
    # Static method: doesn't need class or instance, just namespace
    @staticmethod
    def is_valid_distance(distance):
        """Check if distance is valid (utility function)"""
        return 0 < distance < 1000
    
    @staticmethod
    def convert_km_to_miles(km):
        return km / 1.60934

# Test
print("Class Methods, Static Methods, and Properties:")
print("=" * 60)

# Instance method
trip1 = Trip(distance=10.0, multiplier=1.5)
print(f"Trip 1 fare: ${trip1.calculate_fare():.2f}")

# Property
print(f"Trip 1 fare (via property): ${trip1.fare:.2f}")
trip1.fare = 15.0
print(f"Trip 1 fare (after setting): ${trip1.fare:.2f}")

# Class method (alternative constructor)
trip2 = Trip.from_miles(5.0)  # 5 miles = ~8 km
print(f"\nTrip 2 from 5 miles: {trip2.distance:.2f} km")

# Class method (modify class variable)
print(f"\nBase fare before: ${Trip.base_fare}")
Trip.set_base_fare(3.00)
print(f"Base fare after: ${Trip.base_fare}")

# Static method
print(f"\nIs 50 km valid? {Trip.is_valid_distance(50)}")
print(f"Is 2000 km valid? {Trip.is_valid_distance(2000)}")
print(f"10 km in miles: {Trip.convert_km_to_miles(10):.2f}")

print(f"\nTotal trips created: {Trip.total_trips}")


## Question 49: Three Sum (Extension of Two Sum)

**Difficulty:** Medium  
**Topic:** Arrays, Two Pointers  
**Use Case:** Finding triplets, combination problems

Find all unique triplets that sum to zero.


In [ ]:
def three_sum(nums, target=0):
    """
    Find all unique triplets that sum to target.
    
    Time Complexity: O(n²)
    Space Complexity: O(1) excluding output
    
    Key Concept: Sort array, fix one element, use two pointers for rest
    """
    nums.sort()
    result = []
    n = len(nums)
    
    for i in range(n - 2):
        # Skip duplicates for first element
        if i > 0 and nums[i] == nums[i - 1]:
            continue
        
        left, right = i + 1, n - 1
        
        while left < right:
            current_sum = nums[i] + nums[left] + nums[right]
            
            if current_sum == target:
                result.append([nums[i], nums[left], nums[right]])
                
                # Skip duplicates
                while left < right and nums[left] == nums[left + 1]:
                    left += 1
                while left < right and nums[right] == nums[right - 1]:
                    right -= 1
                
                left += 1
                right -= 1
            elif current_sum < target:
                left += 1
            else:
                right -= 1
    
    return result

# Test
print("Three Sum:")
print("=" * 60)
nums = [-1, 0, 1, 2, -1, -4]
result = three_sum(nums)
print(f"Array: {nums}")
print(f"Triplets that sum to 0: {result}")

# Application: Find three trips with combined distance equal to target
trip_distances = [5.2, 3.1, 8.7, 2.5, 6.3, 4.9]
target_distance = 15.0
# Convert to integers for three_sum
int_distances = [int(d * 10) for d in trip_distances]
int_target = int(target_distance * 10)
result_trips = three_sum(int_distances, int_target)
print(f"\nTrip distances: {trip_distances}")
print(f"Triplets with combined distance ~{target_distance}:")
for triplet in result_trips:
    indices = [int_distances.index(x) for x in triplet]
    print(f"  Trips {indices}: {[trip_distances[i] for i in indices]}")


## Question 50: Python Exception Handling and Custom Exceptions

**Difficulty:** Easy  
**Topic:** Python Fundamentals  
**Use Case:** Error handling, robust code, custom error types

Implement custom exceptions and proper error handling.


In [ ]:
# Custom Exceptions
class InvalidTripException(Exception):
    """Custom exception for invalid trip data"""
    pass

class InsufficientDataException(Exception):
    """Custom exception for insufficient data"""
    def __init__(self, message, required_count, actual_count):
        self.message = message
        self.required_count = required_count
        self.actual_count = actual_count
        super().__init__(self.message)

class ValidationError(Exception):
    """Base exception for validation errors"""
    pass

class DistanceError(ValidationError):
    """Distance validation error"""
    pass

# Function with exception handling
def validate_trip(distance, duration):
    """
    Validate trip data with custom exceptions.
    """
    if distance <= 0:
        raise DistanceError(f"Distance must be positive, got {distance}")
    
    if duration <= 0:
        raise ValidationError(f"Duration must be positive, got {duration}")
    
    if distance > 1000:
        raise InvalidTripException(f"Distance too large: {distance} km")
    
    return True

def process_trips(trips):
    """Process trips with error handling"""
    valid_trips = []
    errors = []
    
    for trip in trips:
        try:
            validate_trip(trip['distance'], trip['duration'])
            valid_trips.append(trip)
        except DistanceError as e:
            errors.append(f"Distance error: {e}")
        except ValidationError as e:
            errors.append(f"Validation error: {e}")
        except InvalidTripException as e:
            errors.append(f"Invalid trip: {e}")
        except Exception as e:
            errors.append(f"Unexpected error: {e}")
    
    return valid_trips, errors

# Test
print("Exception Handling:")
print("=" * 60)

# Test valid trip
try:
    validate_trip(5.2, 15)
    print("✓ Valid trip processed")
except Exception as e:
    print(f"✗ Error: {e}")

# Test invalid distance
try:
    validate_trip(-5, 15)
except DistanceError as e:
    print(f"✗ Caught DistanceError: {e}")
except Exception as e:
    print(f"✗ Caught exception: {e}")

# Test processing multiple trips
trips = [
    {'distance': 5.2, 'duration': 15},
    {'distance': -3.1, 'duration': 10},  # Invalid
    {'distance': 1500, 'duration': 60},  # Invalid
    {'distance': 8.7, 'duration': 20},
]

valid, errors = process_trips(trips)
print(f"\nValid trips: {len(valid)}")
print(f"Errors: {len(errors)}")
for error in errors:
    print(f"  - {error}")
